In [1490]:
import pandas as pd
import os, re
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine, types
from sqlalchemy import text 
from openpyxl import load_workbook

In [1491]:
from dotenv import dotenv_values

config = dotenv_values()

# define variables for the login
pg_user = config['POSTGRES_USER']  # align the key label with your .env file !
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

In [1492]:
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [1493]:
my_schema = 'team_5' # update it to your schema

with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {my_schema};'))

In [1494]:
df_1= pd.read_csv('../data/Raw/World_Marriage_Dataset.csv')

In [1495]:
df_1.drop(columns=["Sr.No."], inplace=True)

In [1496]:
df_1.columns = df_1.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1497]:
df_1.rename(columns={
    "agegroup": "age_group",
    "maritalstatus": "marital_status",
    "dataprocess": "data_process",
    "Data Collection (Start Year)": "data_collection_start_year",
    "Data Collection (End Year)": "data_collection_end_year",
    "Data Source": "data_source",
    "Country": "country",
    "Sex": "sex"
}, inplace=True)

In [1498]:
df_1.drop_duplicates(inplace=True)
df_1.dropna(inplace=True)

df_1['data_collection_start_year'] = df_1['data_collection_start_year'].astype(str).str.replace(',', '').astype(int)
df_1['data_collection_end_year'] = df_1['data_collection_end_year'].astype(str).str.replace(',', '').astype(int)

In [1499]:
df_info = pd.DataFrame({
    'datatypes': df_1.dtypes,
    'null_count': df_1.isnull().sum(),
    'unique_count': df_1.nunique()
})
print(df_info)

                           datatypes  null_count  unique_count
country                       object           0           235
age_group                     object           0            63
sex                           object           0             2
marital_status                object           0            35
data_process                  object           0             6
data_collection_start_year     int32           0            62
data_collection_end_year       int32           0            60
data_source                   object           0            15


In [1500]:
#df_1.to_csv("cleaned_world_marriage.csv", index=False)

In [1501]:
#df_1.to_sql('world_marriage', engine, if_exists='replace', index=False)

In [1502]:
df_2 = pd.read_csv('../data/Raw/age-at-marriage-women.csv')

In [1503]:
df_2.columns = df_2.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1504]:
df_2 = df_2.drop(columns=['1005564annotations'])

df_2.rename(columns={
    "entity": "country",   
}, inplace=True)

In [1505]:
df_2.drop_duplicates(inplace=True)
df_2.dropna(inplace=True)


In [1506]:
df_2['year'] = df_2['year'].astype(str).str.replace(',', '').astype(int)

In [1507]:
df_info = pd.DataFrame({
    'datatypes': df_2.dtypes,
    'null_count': df_2.isnull().sum(),
    'unique_count': df_2.nunique()
})
print(df_info)

                                    datatypes  null_count  unique_count
country                                object           0            41
code                                   object           0            41
year                                    int32           0            32
mean_age_of_women_at_first_marriage   float64           0           179


In [1508]:
#df_2.to_csv("cleaned_age_at_marriage_women.csv", index=False)

In [1509]:
#df_2.to_sql('age_at_marriage_women', engine, if_exists='replace', index=False)

In [1510]:
df_3= pd.read_csv('../data/Raw/marriage-rate-per-1000-inhabitants.csv')

In [1511]:
df_3.columns = df_3.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1512]:
df_3.rename(columns={
    "entity": "country",   
}, inplace=True)

In [1513]:
df_3['year'] = df_3['year'].astype(str).str.replace(',', '').astype(int)

In [1514]:
df_3.drop_duplicates(inplace=True)
df_3.dropna(inplace=True)


In [1515]:
df_info = pd.DataFrame({
    'datatypes': df_3.dtypes,
    'null_count': df_3.isnull().sum(),
    'unique_count': df_3.nunique()
})
print(df_info)

                                              datatypes  null_count  \
country                                          object           0   
code                                             object           0   
year                                              int32           0   
crude_marriage_rate_marriages_per_1000_people   float64           0   

                                               unique_count  
country                                                  45  
code                                                     45  
year                                                    127  
crude_marriage_rate_marriages_per_1000_people           109  


In [1516]:
#df_3.to_csv("cleaned_marriage-rate-per-1000-inhabitants.csv", index=False)

In [1517]:
#df_3.to_sql('married_rate_per_1000', engine, if_exists='replace', index=False)

In [1518]:
df_4= pd.read_csv('../data/Raw/marriage-rates-in-1990-vs-2020.csv')

In [1519]:
df_4.columns = df_4.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1520]:
df_4 = df_4.drop(columns=['worldregionsaccordingtoowid'])

df_4.rename(columns={
    "crudemarriageratemarriagesper1000people": "crude_marriage_rate",
    "crudemarriageratemarriagesper1000people1": "crude_marriage_rate_people1",
    "year1": "year_1",
    "entity": "country"
}, inplace=True)

In [1521]:
df_4.drop_duplicates(inplace=True)
df_4.dropna(inplace=True)

In [1522]:
df_4['year_1'] = pd.to_numeric(df_4['year_1'], errors='coerce').astype('Int64')

In [1523]:
df_info = pd.DataFrame({
    'datatypes': df_4.dtypes,
    'null_count': df_4.isnull().sum(),
    'unique_count': df_4.nunique()
})
print(df_info)

                            datatypes  null_count  unique_count
country                        object           0            38
code                           object           0            38
year                            int64           0            61
crude_marriage_rate           float64           0           101
crude_marriage_rate_people1   float64           0            28
year_1                          Int64           0             1


In [1524]:
#df_4.to_csv("cleaned_marriage-rates-in-1990-vs-2020.csv", index=False)

In [1525]:
#df_4.to_sql('marriage_rates_in_1990_vs_2020', engine, if_exists='replace', index=False)

In [1526]:
df_5 = pd.read_csv('../data/Raw/share-of-births-outside-marriage.csv')

In [1527]:
df_5.columns = df_5.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1528]:

df_5.rename(columns={
    "shareofbirthsoutsideofmarriageofallbirths": "share_of_births_outside_of_marriage",
    "entity": "country"
}, inplace=True)

df_5.drop_duplicates(inplace=True)
df_5.dropna(inplace=True)

In [1529]:
df_info = pd.DataFrame({
    'datatypes': df_5.dtypes,
    'null_count': df_5.isnull().sum(),
    'unique_count': df_5.nunique()
})
print(df_info)

                                    datatypes  null_count  unique_count
country                                object           0            42
code                                   object           0            42
year                                    int64           0            62
share_of_births_outside_of_marriage   float64           0           610


In [1530]:
#df_5.to_csv("cleaned_share-of-births-outside-marriage.csv", index=False)

In [1531]:
#df_5.to_sql('share_of_births_outside_marriage', engine, if_exists='replace', index=False)

In [1532]:
df_6 = pd.read_csv('../data/Raw/share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv')
df_6

,Entity,Code,Year,Proportions of men or women who had ever married by a certain age for 1900 birth cohort,Proportions of men or women who had ever married by a certain age for 1920 birth cohort,Proportions of men or women who had ever married by a certain age for 1940 birth cohort,Proportions of men or women who had ever married by a certain age for 1960 birth cohort,Proportions of men or women who had ever married by a certain age for 1970 birth cohort,Proportions of men or women who had ever married by a certain age for 1980 birth cohort,Proportions of men or women who had ever married by a certain age for 1990 birth cohort,Proportions of men or women who had ever married by a certain age for 2000 birth cohort
0,Men,NaN,17,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0
1,Men,NaN,18,0.1,0.1,0.4,0.6,0.1,0.0,0.0,0.0
2,Men,NaN,19,0.8,0.6,2.0,2.5,0.7,0.3,0.1,0.0
3,Men,NaN,20,2.4,2.2,6.0,6.2,1.9,0.7,0.3,0.1
4,Men,NaN,21,6.1,7.4,13.6,11.9,3.9,1.4,0.6,0.2
...,...,...,...,...,...,...,...,...,...,...,...
63,Women,NaN,46,84.5,91.6,95.5,86.9,75.0,NaN,NaN,NaN
64,Women,NaN,47,84.8,91.7,95.6,87.0,75.4,NaN,NaN,NaN
65,Women,NaN,48,85.0,91.8,95.6,87.2,75.7,NaN,NaN,NaN
66,Women,NaN,49,85.2,91.9,95.7,87.3,76.0,NaN,NaN,NaN


In [1533]:
df_6.columns = df_6.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

df_6.drop_duplicates(inplace=True)

df_6.head()

,entity,code,year,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort
0,Men,NaN,17,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0
1,Men,NaN,18,0.1,0.1,0.4,0.6,0.1,0.0,0.0,0.0
2,Men,NaN,19,0.8,0.6,2.0,2.5,0.7,0.3,0.1,0.0
3,Men,NaN,20,2.4,2.2,6.0,6.2,1.9,0.7,0.3,0.1
4,Men,NaN,21,6.1,7.4,13.6,11.9,3.9,1.4,0.6,0.2


In [1534]:
df_6 = df_6.drop(columns=['code','proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort'])

df_6.rename(columns={
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort": "1900_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort": "1920_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort": "1940_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort": "1960_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort": "1970_birthcohort",
    "entity": "sex"
}, inplace=True)
df_6

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
0,Men,17,0.0,0.0,0.0,0.1,0.0
1,Men,18,0.1,0.1,0.4,0.6,0.1
2,Men,19,0.8,0.6,2.0,2.5,0.7
3,Men,20,2.4,2.2,6.0,6.2,1.9
4,Men,21,6.1,7.4,13.6,11.9,3.9
...,...,...,...,...,...,...,...
63,Women,46,84.5,91.6,95.5,86.9,75.0
64,Women,47,84.8,91.7,95.6,87.0,75.4
65,Women,48,85.0,91.8,95.6,87.2,75.7
66,Women,49,85.2,91.9,95.7,87.3,76.0


In [1535]:
df_6.dropna(inplace=True)
df_6.describe

<bound method NDFrame.describe of       sex  year  1900_birthcohort  1920_birthcohort  1940_birthcohort  \
0     Men    17               0.0               0.0               0.0   
1     Men    18               0.1               0.1               0.4   
2     Men    19               0.8               0.6               2.0   
3     Men    20               2.4               2.2               6.0   
4     Men    21               6.1               7.4              13.6   
..    ...   ...               ...               ...               ...   
63  Women    46              84.5              91.6              95.5   
64  Women    47              84.8              91.7              95.6   
65  Women    48              85.0              91.8              95.6   
66  Women    49              85.2              91.9              95.7   
67  Women    50              85.4              92.0              95.7   

    1960_birthcohort  1970_birthcohort  
0                0.1               0.0  
1      

In [1536]:
df_info = pd.DataFrame({
    'datatypes': df_6.dtypes,
    'null_count': df_6.isnull().sum(),
    'unique_count': df_6.nunique()
})
print(df_info)

                 datatypes  null_count  unique_count
sex                 object           0             2
year                 int64           0            34
1900_birthcohort   float64           0            66
1920_birthcohort   float64           0            61
1940_birthcohort   float64           0            62
1960_birthcohort   float64           0            67
1970_birthcohort   float64           0            65


In [1537]:
df_6.sample(12)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
57,Women,40,82.4,90.5,95.0,85.3,72.0
15,Men,32,79.3,82.2,86.5,68.8,48.2
46,Women,29,68.7,80.4,89.8,74.5,52.4
39,Women,22,24.3,39.3,57.5,40.6,18.2
35,Women,18,0.4,1.6,4.6,4.6,1.3
21,Men,38,88.1,89.0,90.5,76.6,61.3
60,Women,43,83.7,91.1,95.3,86.2,73.7
47,Women,30,71.6,82.7,90.9,76.7,55.8
49,Women,32,75.5,85.8,92.4,79.9,61.0
31,Men,48,92.5,91.7,92.3,81.3,69.9


In [1538]:
#df_6.to_csv("cleaned_share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv", index=False)

In [1539]:
#df_6.to_sql('men_in_england_and_wales_married_by_age', engine, if_exists='replace', index=False)

In [1540]:
df_7 = pd.read_csv('../data/Raw/share-of-births-outside-marriage.csv')

In [1541]:
df_7.columns = df_7.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1542]:
df_7.rename(columns={
    "shareofsingleparenthouseholds": "share_of_single_parent_households",
    "entity": "country"
}, inplace=True)

df_7.drop_duplicates(inplace=True)
df_7.dropna(inplace=True)
df_7.sample(5)

,country,code,year,shareofbirthsoutsideofmarriageofallbirths
429,Cyprus,CYP,2005,4.4
1999,Switzerland,CHE,2014,21.7
1359,Netherlands,NLD,1967,2.1
479,Czechia,CZE,1995,15.6
2045,United Kingdom,GBR,1989,26.6


In [1543]:
df_info = pd.DataFrame({
    'datatypes': df_7.dtypes,
    'null_count': df_7.isnull().sum(),
    'unique_count': df_7.nunique()
})
print(df_info)

                                          datatypes  null_count  unique_count
country                                      object           0            42
code                                         object           0            42
year                                          int64           0            62
shareofbirthsoutsideofmarriageofallbirths   float64           0           610


In [1544]:
#df_7.to_csv("cleaned_share-of-single-parent-households.csv", index=False)

In [1545]:
#df_7.to_sql('single_parent_households', engine, if_exists='replace', index=False)

In [1546]:
df_8 = pd.read_csv('../data/Raw/share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv')

In [1547]:
df_8.columns = df_8.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1548]:
df_8['code'] = df_8['code'].fillna('GBR')
df_8.sample(5)

,entity,code,year,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort
64,Women,GBR,47,84.8,91.7,95.6,87.0,75.4,NaN,NaN,NaN
35,Women,GBR,18,0.4,1.6,4.6,4.6,1.3,0.4,0.1,0.0
60,Women,GBR,43,83.7,91.1,95.3,86.2,73.7,63.7,NaN,NaN
58,Women,GBR,41,82.9,90.7,95.1,85.7,72.7,62.0,NaN,NaN
48,Women,GBR,31,73.9,84.5,91.7,78.4,58.7,42.0,30.2,NaN


In [1549]:
df_8 = df_8.drop(columns=['code','proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort'])

df_8.rename(columns={
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort": "1900_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort": "1920_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort": "1940_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort": "1960_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort": "1970_birthcohort",
    "entity": "sex"
}, inplace=True)

df_8.drop_duplicates(inplace=True)
df_8.dropna(inplace=True)
df_8.sample(5)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
28,Men,45,91.8,91.3,92.0,80.5,68.2
23,Men,40,89.6,89.9,91.1,78.0,64.0
54,Women,37,80.6,89.4,94.4,84.1,69.4
45,Women,28,65.0,77.4,88.4,72.0,48.8
13,Men,30,72.9,76.4,83.3,63.9,41.4


In [1550]:
df_info = pd.DataFrame({
    'datatypes': df_8.dtypes,
    'null_count': df_8.isnull().sum(),
    'unique_count': df_8.nunique()
})
print(df_info)

                 datatypes  null_count  unique_count
sex                 object           0             2
year                 int64           0            34
1900_birthcohort   float64           0            66
1920_birthcohort   float64           0            61
1940_birthcohort   float64           0            62
1960_birthcohort   float64           0            67
1970_birthcohort   float64           0            65


In [1551]:
#df_8.to_csv("cleaned_share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv", index=False)

In [1552]:
#df_8.to_sql('women_in_england_and_wales_married_by_age', engine, if_exists='replace', index=False)

In [1553]:
#pip install openpyxl pywin32

In [1554]:
df_excel_1 = pd.read_excel('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx')

In [1555]:
#all_sheets = pd.read_excel('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx', sheet_name=None)

In [1556]:
xls_1 = pd.ExcelFile('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx')
print(xls_1.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'MARITAL_STATUS_BY_AGE', 'CURRENTLY MARRIED', 'EVER_MARRIED', 'SMAM']


In [1557]:
excel_1 = '../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx'

# Output directory (make sure it exists)
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

# List of sheets you want to extract
sheets_to_extract = ['MARITAL_STATUS_BY_AGE', 'CURRENTLY MARRIED', 'EVER_MARRIED', 'SMAM']

In [1558]:
"""for sheet in sheets_to_extract:
    # Read just this sheet into a DataFrame
    df_excel_1 = pd.read_excel(excel_1, sheet_name=sheet)
    
    # Optional: Clean the filename (replace spaces with underscores, etc.)
    csv_name = sheet.replace(' ', '_').lower() + '.csv'
    csv_path = os.path.join(output_dir, csv_name)
    
    # Save the DataFrame as CSV
    df_excel_1.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")
"""

'for sheet in sheets_to_extract:\n    # Read just this sheet into a DataFrame\n    df_excel_1 = pd.read_excel(excel_1, sheet_name=sheet)\n    \n    # Optional: Clean the filename (replace spaces with underscores, etc.)\n    csv_name = sheet.replace(\' \', \'_\').lower() + \'.csv\'\n    csv_path = os.path.join(output_dir, csv_name)\n    \n    # Save the DataFrame as CSV\n    df_excel_1.to_csv(csv_path, index=False)\n    print(f"Saved: {csv_path}")\n'

In [1559]:
xls_2 = pd.ExcelFile('../data/Raw/undesa_pd_2019_world_fertility_dataset.xlsx')
print(xls_2.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'FERTILITY INDICATORS']


In [1560]:
excel_2 = '../data/Raw/undesa_pd_2019_world_fertility_dataset.xlsx'
sheet_name = 'FERTILITY INDICATORS'
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

df_excel_2 = pd.read_excel(excel_2, sheet_name=sheet_name)


In [1561]:
"""csv_name = sheet_name.replace(' ', '_').lower() + '.csv'
csv_path = os.path.join(output_dir, csv_name)
df_excel_2.to_csv(csv_path, index=False)
print(f"Saved: {csv_path}")
"""

'csv_name = sheet_name.replace(\' \', \'_\').lower() + \'.csv\'\ncsv_path = os.path.join(output_dir, csv_name)\ndf_excel_2.to_csv(csv_path, index=False)\nprint(f"Saved: {csv_path}")\n'

In [1562]:
xls_3 = pd.ExcelFile('../data/Raw/undesa_pd_ds_1970-2030_fp_rev-2024_rev.xlsx')
print(xls_3.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'Countries', 'Regions']


In [1563]:
excel_3 = '../data/Raw/undesa_pd_ds_1970-2030_fp_rev-2024_rev.xlsx'
sheets_to_extract = ['Countries', 'Regions']
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)


In [1564]:
"""
for sheet in sheets_to_extract:
    df = pd.read_excel(excel_3, sheet_name=sheet)
    csv_name = sheet.replace(' ', '_').lower() + '.csv'
    csv_path = os.path.join(output_dir, csv_name)
    df.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")

"""

'\nfor sheet in sheets_to_extract:\n    df = pd.read_excel(excel_3, sheet_name=sheet)\n    csv_name = sheet.replace(\' \', \'_\').lower() + \'.csv\'\n    csv_path = os.path.join(output_dir, csv_name)\n    df.to_csv(csv_path, index=False)\n    print(f"Saved: {csv_path}")\n\n'

In [1565]:
df_9 = pd.read_csv('../data/Raw/unpopulation_dataportal_20250728095844.csv')
df_9.sample(5)

,IndicatorId,IndicatorName,IndicatorShortName,Source,SourceYear,Author,LocationId,Location,Iso2,Iso3,...,AgeStart,AgeEnd,Age,CategoryId,Category,EstimateTypeId,EstimateType,EstimateMethodId,EstimateMethod,Value
6390,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,222,El Salvador,SV,SLV,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,57.67
22203,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,768,Togo,TG,TGO,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,72.10
13982,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,474,Martinique,MQ,MTQ,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,33.11
10284,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,360,Indonesia,ID,IDN,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,70.99
9495,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,328,Guyana,GY,GUY,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,57.87


In [1566]:
df_9.columns = df_9.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_9.sample(5)

,indicatorid,indicatorname,indicatorshortname,source,sourceyear,author,locationid,location,iso2,iso3,...,agestart,ageend,age,categoryid,category,estimatetypeid,estimatetype,estimatemethodid,estimatemethod,value
21565,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,748,Eswatini,SZ,SWZ,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,44.40
15421,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,524,Nepal,NP,NPL,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,76.01
9440,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,328,Guyana,GY,GUY,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,55.54
21265,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,729,Sudan,SD,SDN,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,3,Projection,61.63
6122,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,212,Dominica,DM,DMA,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,48.48


In [1567]:
df_9 = df_9.drop(columns=['indicatorid','indicatorshortname',
    'source',
    'author', 'locationid', 'iso2','estimatetypeid','category','categoryid','agestart','ageend','author','ageid', 'estimatetype','variantid','sexid','timeid'])

df_9.rename(columns={
    "sourceyear": "year",
    "location": "country",
    "estimatemethodid": "estimate_method",
    "iso3": "code",
}, inplace=True)



In [1568]:
df_9.drop_duplicates(inplace=True)
df_9.dropna(inplace = True)

df_9

,indicatorname,year,country,code,time,variant,sex,age,estimate_method,estimatemethod,value
0,Currently married (Percent),2024,Afghanistan,AFG,1970,Median,Female,15-49,2,Interpolation,80.94
2,Currently married (Percent),2024,Afghanistan,AFG,1971,Median,Female,15-49,2,Interpolation,80.90
4,Currently married (Percent),2024,Afghanistan,AFG,1972,Median,Female,15-49,2,Interpolation,80.87
6,Currently married (Percent),2024,Afghanistan,AFG,1973,Median,Female,15-49,2,Interpolation,80.84
8,Currently married (Percent),2024,Afghanistan,AFG,1974,Median,Female,15-49,2,Interpolation,80.53
...,...,...,...,...,...,...,...,...,...,...,...
25078,Currently married (Percent),2024,Zambia,ZMB,2021,Median,Female,15-49,3,Projection,54.31
25080,Currently married (Percent),2024,Zambia,ZMB,2022,Median,Female,15-49,3,Projection,53.82
25082,Currently married (Percent),2024,Zambia,ZMB,2023,Median,Female,15-49,3,Projection,53.35
25084,Currently married (Percent),2024,Zambia,ZMB,2024,Median,Female,15-49,3,Projection,52.91


In [1569]:
df_info = pd.DataFrame({
    'datatypes': df_9.dtypes,
    'null_count': df_9.isnull().sum(),
    'unique_count': df_9.nunique()
})
print(df_info)

                datatypes  null_count  unique_count
indicatorname      object           0             1
year                int64           0             1
country            object           0           224
code               object           0           224
time                int64           0            56
variant            object           0             1
sex                object           0             1
age                object           0             1
estimate_method     int64           0             2
estimatemethod     object           0             2
value             float64           0          3867


In [1570]:
#df_9.to_csv("cleaned_unpopulation_dataportal.csv", index=False)

In [1571]:
#df_9.to_sql('unpopulation_dataportal', engine, if_exists='replace', index=False)

In [1572]:
df_10 = pd.read_csv('../data/processed/countries_un.csv',  header=5, low_memory=False)

In [1573]:
df_10.columns = (
    df_10.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
)
df_10.sample(10)

,countryorarea,isocode,indicator,year,agegroup,percentage,number,dataprocess
10791,Bermuda,60,Married or in-union women,2022,15-49,47.253411,6.399766,Projection
38829,Ethiopia,231,Married or in-union women,2044,40-44,86.150626,5172.445222,Projection
54363,Guatemala,320,Married or in-union women,2042,30-34,65.328324,605.349236,Projection
54675,Guinea,324,Married or in-union women,2000,30-34,95.814435,247.975903,Estimate
3978,Azerbaijan,31,Married or in-union women,1981,25-29,78.321579,204.771377,Estimate
51153,Greece,300,Married or in-union women,2046,20-24,4.708345,9.268588,Projection
52821,Guadeloupe,312,Married or in-union women,2011,40-44,52.365909,9.193883,Estimate
16541,Brunei Darussalam,96,Married or in-union women,2012,40-44,80.221802,12.041293,Projection
74039,Libya,434,Married or in-union women,1990,15-49,52.849501,497.521235,Estimate
58000,Hungary,348,Married or in-union women,2011,15-19,4.252439,12.212749,Estimate


In [1574]:
df_10.rename(columns={
    "dataprocess": "data_process",
}, inplace=True)

df_10.drop_duplicates(inplace=True)
df_10.sample(5)

,countryorarea,isocode,indicator,year,agegroup,percentage,number,data_process
97399,Micronesia (Fed. States of),583,Married or in-union women,1994,15-49,51.041554,12.940310,Estimate
6549,Bahamas,44,Married or in-union women,1978,40-44,65.699174,3.737297,Estimate
142243,Uzbekistan,860,Married or in-union women,2011,30-34,89.926923,958.554454,Estimate
127097,Syrian Arab Republic,760,Married or in-union women,1981,20-24,59.587097,229.121623,Estimate
93509,New Zealand,554,Married or in-union women,1994,40-44,78.053440,98.633791,Estimate


In [1575]:
for col in ['percentage', 'number']:
    if col in df_10.columns:
        df_10[col] = (
            df_10[col]
            .astype(str)
            .str.replace(',', '.', regex=False)
            .str.extract(r'([-+]?[0-9]*\.?[0-9]+)', expand=False)
            .astype(float)
            .round(2)
        )

In [1576]:
unnamed_cols = [col for col in df_10.columns if 'unnamed' in col.lower()]
df_10.drop(columns=unnamed_cols, inplace=True)

In [1577]:
df_10.dropna(inplace=True)

In [1578]:
df_info = pd.DataFrame({
    'datatypes': df_10.dtypes,
    'null_count': df_10.isnull().sum(),
    'unique_count': df_10.nunique()
})
print(df_info)

              datatypes  null_count  unique_count
countryorarea    object           0           225
isocode           int64           0           225
indicator        object           0             1
year              int64           0            81
agegroup         object           0             8
percentage      float64           0          9667
number          float64           0         65394
data_process     object           0             2


In [1579]:
df_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145800 entries, 0 to 145799
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   countryorarea  145800 non-null  object 
 1   isocode        145800 non-null  int64  
 2   indicator      145800 non-null  object 
 3   year           145800 non-null  int64  
 4   agegroup       145800 non-null  object 
 5   percentage     145800 non-null  float64
 6   number         145800 non-null  float64
 7   data_process   145800 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 8.9+ MB


In [1580]:
#df_10.to_csv("cleaned_countries_1970_2025_un.csv", index=False)

In [1581]:
#df_10.to_sql('countries_1970_2025_un', engine, if_exists='replace', index=False)

In [1582]:
df_11 = pd.read_csv('../data/processed/currently_married_un.csv',  header=2, low_memory=False)

In [1583]:
df_11.sample(8)

,Country or area,ISO code,YearStart,YearEnd,Sex,AgeGroup,AgeStart,AgeEnd,DataValue,DataProcess,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Data,Note on Country and Population
1386,Australia,36,1997,1997,Men,[25-29],25,29,34.70,Estimate,1997 Estimate,2037,Australia 1997 Estimate,UNSD,NaN,NaN,NaN
35760,Norway,578,1980,1980,Women,[25-29],25,29,70.22,Estimate,1980 Estimate,2180,Norway 1980 Estimate,UNSD,NaN,NaN,NaN
8406,"China, Macao SAR",446,2006,2006,Men,[50-54],50,54,91.78,Census,2006 Census,3177,"China, Macao (SAR) 2006 Census",UNSD,NaN,NaN,NaN
44543,Slovenia,705,2013,2013,Women,[40-44],40,44,58.90,Estimate,2013 Estimate,2218,Slovenia 2013 Estimate,UNSD,NaN,NaN,NaN
47031,Sweden,752,1996,1996,Women,[45-49],45,49,64.16,Estimate,1996 Estimate,2227,Sweden 1996 Estimate,UNSD,NaN,NaN,NaN
11805,Denmark,208,2009,2009,Men,[60-64],60,64,72.71,Estimate,2009 Estimate,2081,Denmark 2009 Estimate,UNSD,NaN,Based on data compiled from registers.,Excluding Faeroe Islands and Greenland shown s...
35077,Niger,562,2006,2006,Women,[20-24],20,24,86.40,Survey,2006 DHS,1736,Niger 2006 Demographic and Health Survey,DHS_STATcompiler,1.0,NaN,NaN
5846,Cabo Verde,132,2000,2000,Men,[30-34],30,34,59.40,Census,2000 Census,1219,Cape Verde 2000 Census,National statistics,1.0,NaN,NaN


In [1584]:
df_11.columns = (
    df_11.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
    )
df_11.sample(8)

,countryorarea,isocode,yearstart,yearend,sex,agegroup,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteondata,noteoncountryandpopulation
24565,Israel,376,2013,2013,Women,[40-44],40,44,74.57,Estimate,2013 Estimate,2127,Israel 2013 Estimate,UNSD,NaN,NaN,NaN
14109,Falkland Islands (Malvinas),238,2016,2016,Men,[15-19],15,19,1.64,Census,2016 Census,7296,Falkland Islands (Malvinas) 2016 Census,National statistics,1.0,NaN,NaN
47633,Switzerland,756,1986,1986,Women,[10-14],10,14,0.00,Estimate,1986 Estimate,2228,Switzerland 1986 Estimate,UNSD,NaN,NaN,NaN
43165,Seychelles,690,1994,1994,Men,[35-39],35,39,71.95,Census,1994 Census,2562,Seychelles 1994 Census,UNSD,1.0,NaN,NaN
18957,Guinea,324,2012,2012,Men,[30-34],30,34,79.50,Survey,2012 DHS,5422,Guinea 2012 Demographic and Health Survey,DHS_STATcompiler,1.0,NaN,NaN
4851,Bulgaria,100,1985,1985,Women,[15-19],15,19,15.83,Census,1985 Census,846,Bulgaria 1985 Census,UNSD,1.0,NaN,NaN
16016,France,250,1988,1988,Men,[25-29],25,29,45.94,Estimate,1988 Estimate,2094,France 1988 Estimate,UNSD,NaN,NaN,Excluding diplomatic personnel outside the cou...
15241,Finland,246,2011,2011,Men,[70-74],70,74,70.81,Estimate,2011 Estimate,2093,Finland 2011 Estimate,UNSD,1.0,NaN,Excluding Åland Islands.


In [1585]:
df_11 = df_11.drop(columns = ['datacataloglongname', 'datacatalogid', 'yearstart' , 'yearend', 'noteondata', 'noteoncountryandpopulation', 'including_consensual_unions'])

df_11.rename(columns={
    "agestart": "age_start",
    "countryorarea": "country",
    "datasource": "data_source",
    "datavalue" : "data_value"
}, inplace=True)

df_11.sample(10)

,country,isocode,sex,agegroup,age_start,ageend,data_value,dataprocess,datacatalogshortname,data_source
39361,Republic of Moldova,498,Men,[50-54],50,54,91.02,Survey,2000 MICS_HH,MICS_HH
37147,Pakistan,586,Men,[30-34],30,34,80.87,Survey,2017 DHS,DHS_HH
14978,Finland,246,Women,[75+],75,999,19.91,Census,2000 Census,UNSD
4905,Bulgaria,100,Women,[45-49],45,49,81.38,Census,2001 Census,UNSD
52202,Vanuatu,548,Women,[25-29],25,29,86.05,Survey,2007 MICS,MICS
36886,Pakistan,586,Women,[35-39],35,39,94.75,Census,1981 Census,UNSD
49695,Trinidad and Tobago,780,Women,[65+],65,999,31.29,Census,2000 Census,UNSD
40934,Saint Vincent and the Grenadines,670,Women,[75+],75,999,18.53,Census,2012 Census,National statistics
45705,State of Palestine,275,Women,[15-19],15,19,26.90,Census,1997 Census,UNSD
41647,San Marino,674,Men,[30-34],30,34,76.34,Estimate,1986 Estimate,UNSD


In [1586]:
df_11.drop_duplicates(inplace=True)
df_11.dropna(inplace=True)

In [1587]:
df_info = pd.DataFrame({
    'datatypes': df_11.dtypes,
    'null_count': df_11.isnull().sum(),
    'unique_count': df_11.nunique()
})
print(df_info)

                     datatypes  null_count  unique_count
country                 object           0           233
isocode                  int64           0           230
sex                     object           0             2
agegroup                object           0            23
age_start                int64           0            17
ageend                   int64           0            15
data_value             float64           0          9213
dataprocess             object           0             6
datacatalogshortname    object           0           412
data_source             object           0            15


In [1588]:
#df_11.to_csv("cleaned_currently_married_un.csv", index=False)

In [1589]:
#df_11.to_sql('currently_married_un', engine, if_exists='replace', index=False)

In [1590]:
df_12 = pd.read_csv('../data/processed/ever_married_un.csv', header= 2, low_memory = False)
df_12.head()

,Country or area,ISO code,YearStart,YearEnd,Sex,AgeGroup,AgeStart,AgeEnd,DataValue,DataProcess,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Data,Note on Country and Population
0,Afghanistan,4,1972,1974,Men,[15-19],15,19,7.7,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
1,Afghanistan,4,1972,1974,Men,[20-24],20,24,32.6,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
2,Afghanistan,4,1972,1974,Men,[25-29],25,29,61.4,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
3,Afghanistan,4,1972,1974,Men,[30-34],30,34,83.0,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
4,Afghanistan,4,1972,1974,Men,[35-39],35,39,91.2,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN


In [1591]:
df_12.columns = (
    df_12.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
    )
df_12.sample(8)

,countryorarea,isocode,yearstart,yearend,sex,agegroup,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteondata,noteoncountryandpopulation
7914,Chile,152,1970,1970,Men,[70-74],70,74,90.14,Census,1970 Census,1285,Chile 1970 Census,UNSD,1.0,Based on a sample of census returns. | Data ha...,NaN
42888,Saint Vincent and the Grenadines,670,1991,1991,Women,[35-39],35,39,48.11,Census,1991 Census,377,Saint Vincent and the Grenadines 1991 Census,UNSD,NaN,NaN,NaN
31189,Luxembourg,442,2011,2011,Men,[50-54],50,54,88.91,Census,2011 Census,4832,Luxembourg 2011 Census,Eurostat,1.0,Estimates computed based on data on marital st...,NaN
13196,Dominica,212,2001,2001,Women,[50-54],50,54,60.42,Census,2001 Census,2345,Dominica 2001 Census,UNSD,NaN,NaN,NaN
41781,Romania,642,1995,1995,Women,[70-74],70,74,96.81,Estimate,1995 Estimate,2199,Romania 1995 Estimate,UNSD,NaN,NaN,NaN
32597,Mauritania,478,1976,1977,Men,[25-29],25,29,52.98,Census,1976-1977 Census,1063,Mauritania 1976-1977 Census,UNSD,NaN,NaN,Including nomads.
31906,Maldives,462,2009,2009,Women,[25-29],25,29,89.20,Survey,2009 DHS,4699,Maldives 2009 Demographic and Health Survey,DHS_HH,NaN,NaN,NaN
26923,Japan,392,1975,1975,Men,[75+],75,999,98.81,Census,1975 Census,1608,Japan 1975 Census,UNSD,NaN,NaN,Excluding diplomatic personnel outside the cou...


In [1592]:
df_12 = df_12.drop(columns = ['yearstart', 'yearend', 'datacatalogshortname', 'datacatalogid', 'datacataloglongname', 'including_consensual_unions', 'noteondata', 'noteoncountryandpopulation'])

df_12.rename(columns={
    "agestart": "age_start",
    "ageend": "age_end",
    "countryorarea": "country"
}, inplace=True)
df_12.sample(8)

,country,isocode,sex,agegroup,age_start,age_end,datavalue,dataprocess,datasource
4529,Brazil,76,Women,[15-19],15,19,15.54,Census,UNSD
39510,Paraguay,600,Women,[35-39],35,39,81.44,Census,UNSD
10768,Cyprus,196,Men,[25-29],25,29,64.76,Census,UNSD
26223,Italy,380,Women,[45-49],45,49,87.13,Estimate,UNSD
23031,Iceland,352,Women,[20-24],20,24,8.65,Estimate,UNSD
13099,Djibouti,262,Women,[20-24],20,24,14.20,Survey,National statistics
32068,Mali,466,Women,[35-39],35,39,100.00,Survey,DHS_STATcompiler
11004,Czechia,203,Men,[25-29],25,29,68.09,Estimate,UNSD


In [1593]:
df_12.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_12.dtypes,
    'null_count': df_12.isnull().sum(),
    'unique_count': df_12.nunique()
})
print(df_info)

            datatypes  null_count  unique_count
country        object           0           233
isocode         int64           0           230
sex            object           0             2
agegroup       object           0            23
age_start       int64           0            17
age_end         int64           0            15
datavalue     float64           0          8396
dataprocess    object           0             6
datasource     object           0            15


In [1594]:
#df_12.to_csv("cleaned_ever_married_un.csv", index=False)

In [1595]:
#df_12.to_sql('ever_married_un', engine, if_exists= 'replace', index= False)

In [1596]:
df_13 = pd.read_csv('../data/processed/fertility_indicators_un.csv', header=6, low_memory=False)
df_13.head()

,Country or Area,Country or Area Code,Age Group,Indicator,Date,Value,Series,DataType,Data Source Type,Survey Programme,Data Source Inventory ID,Data Source Name,Data Source Name (short),Data Source Start Year,Data Source End Year,Reference,Reference Year
0,Afghanistan,4,[Total],TFR,1964.977051,7.966653,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
1,Afghanistan,4,[Total],TFR,1965.977051,8.212275,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
2,Afghanistan,4,[Total],TFR,1966.977051,8.317603,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
3,Afghanistan,4,[Total],TFR,1967.977051,8.225812,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
4,Afghanistan,4,[Total],TFR,1968.977051,8.068459,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012


In [1597]:
df_13.columns = (df_13.columns
        .str.lower()
        .str.strip()
        .str.replace(' ', '')
        .str.replace('(', '')
        .str.replace(')', '')
        .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
        )

df_13.sample(6)

,countryorarea,countryorareacode,agegroup,indicator,date,value,series,datatype,datasourcetype,surveyprogramme,datasourceinventoryid,datasourcename,datasourcenameshort,datasourcestartyear,datasourceendyear,reference,referenceyear
19400,Cyprus,196,[30-34],ASFR3034,1998.50000,90.8000,"Estimates,Fertility data (Adjusted),HFC-ODE,20...",Fertility data (adjusted),Estimate,Estimate,2078,All sources of estimates,Estimates,1998,1998,European Demographic Observatory (ODE). Data c...,2011
77014,Venezuela (Bolivarian Republic of),862,[20-24],ASFR2024,2002.50000,129.0069,"Register,Computed rate from DYB,DYB,556-135-56",Computed rate from DYB,Register,VR,556,Vital Registration,Register,2002,2002,Demographic Yearbook,2003
1840,Argentina,32,[20-24],ASFR2024,1987.50137,154.1510,VR_Census,Direct,Register,VR,434,Vital Registration,Register,1987,1987,INDEC,2018
557,Albania,8,[40-44],ASFR4044,1979.50000,33.7120,"Estimates,Fertility data (Adjusted),HFC-ODE,20...",Fertility data (adjusted),Estimate,Estimate,2029,All sources of estimates,Estimates,1979,1979,European Demographic Observatory (ODE). Data c...,2011
75237,United States Virgin Islands,850,[Total],TFR,1999.50000,1.9575,"Register, Direct, National Center for Health S...",Direct,Register,VR,553,Vital Registration,Register,1999,1999,National Center for Health Statistics. NVSRs.,
64944,Somalia,706,[20-24],ASFR2024,2000.08400,221.0000,"2002-2003 Reproductive Health Survey,Direct es...",Birth histories,Survey,Survey,2567,Somalia 2002-2003 Reproductive Health Survey,2002-2003 Survey,2002,2003,2002-2003 Reproductive Health Survey Final report,


In [1598]:
df_13 = df_13.drop(columns=['countryorareacode','indicator','datasourceinventoryid','surveyprogramme','series','datasourcename','reference','referenceyear'])

df_13.replace({
    "agegroup": "age_group",
    "countryorarea": "country",
    "datatype": "data_type",
},inplace=True)

In [1599]:
df_13['date'] = df_13['date'].astype(int)
df_13['value'] = df_13['value'].round(2)
df_13.sample(12)

,countryorarea,agegroup,date,value,datatype,datasourcetype,datasourcenameshort,datasourcestartyear,datasourceendyear
59363,Russian Federation,[Total],1951,2.92,Direct,Register,Register,1951,1951
4730,Bahamas,[Total],1990,2.52,Direct,Estimate,Estimates,1990,1990
65875,Spain,[15-19],2006,12.00,Official estimates,Estimate,Estimates,2006,2006
3864,Austria,[15-19],1994,18.91,Official estimates,Estimate,Estimates,1994,1994
42787,Liberia,[40-44],2007,81.00,Direct,Survey,2009 MIS,2008,2009
65454,Spain,[25-29],1958,177.31,Fertility data (adjusted),Estimate,Estimates,1958,1958
58027,Republic of Korea,[30-34],2003,79.70,Direct,Register,Register,2003,2003
40686,Lao People's Dem. Republic,[30-34],1988,275.00,Recent births,Panel,1986-1988 Survey,1986,1988
61450,Senegal,[45-49],1979,63.37,Extrapolated from Truncated Birth Histories,Survey,1997 DHS,1997,1997
33777,India,[45-49],1987,4.43,Extrapolated from Truncated Birth Histories,Survey,1998-1999 NFHS,1998,1999


In [1600]:
df_13.drop_duplicates(inplace=True)
df_13.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_13.dtypes,
    'null_count': df_13.isnull().sum(),
    'unique_count': df_13.nunique()
})
print(df_info)

                    datatypes  null_count  unique_count
countryorarea          object           0           201
agegroup               object           0             8
date                    int32           0            69
value                 float64           0         18752
datatype               object           0            30
datasourcetype         object           0             7
datasourcenameshort    object           0           539
datasourcestartyear     int64           0            69
datasourceendyear      object           0            70


In [1601]:
#df_13.to_csv("../data/Cleaned/cleaned_fertility_indicators_un.csv", index=False)

In [1602]:
#df_13.to_sql('fertility_indicators_un',engine, if_exists='replace', index=False)

In [1603]:
df_14 = pd.read_csv('../data/processed/marital_status_by_age_un.csv', header= 2, low_memory=False)
df_14.head()

,Country or area,ISO code,YearStart,YearEnd,Sex,MaritalStatus,Non-standard_AgeGroups,Series_contains_Non-standard_AgeGroups,AgeGroup,AgeStart,...,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Age groups,Note on Marital Status,Note on Data,Note on Country and Population,Note Other
0,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[15-19],15,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[20-24],20,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[25-29],25,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[30-34],30,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[35-39],35,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN


In [1604]:
df_14.columns= (df_14.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '' , regex=True)  
    )
df_14.sample(5)

,countryorarea,isocode,yearstart,yearend,sex,maritalstatus,nonstandard_agegroups,series_contains_nonstandard_agegroups,agegroup,agestart,...,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteonagegroups,noteonmaritalstatus,noteondata,noteoncountryandpopulation,noteother
16075,Belgium,56,2001,2001,Women,Single,NaN,NaN,[15-19],15,...,2001 Census,3045,Belgium 2001 Census,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
15904,Belgium,56,2000,2000,Men,Married,NaN,NaN,[45-49],45,...,2000 Estimate,2044,Belgium 2000 Estimate,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
167647,Nepal,524,1976,1976,Women,Divorced,NaN,NaN,[50-54],50,...,1976 WFS,818,Nepal 1976 World Fertility Survey,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
24437,Bulgaria,100,1985,1985,Men,Married,NaN,NaN,[65-69],65,...,1985 Census,846,Bulgaria 1985 Census,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
2756,Angola,24,2001,2001,Women,Divorced,NaN,NaN,[35-39],35,...,2001 MICS_HH,1937,Angola 2001 Multiple Indicator Cluster Survey,MICS_HH,NaN,NaN,NaN,NaN,NaN,NaN


In [1605]:
df_14 = df_14.drop(columns=['datacataloglongname', 'noteondata', 'noteoncountryandpopulation','noteonagegroups', 'noteother',
                             'including_consensual_unions','isocode', 'datacatalogid', 'noteonmaritalstatus', 'series_contains_nonstandard_agegroups','nonstandard_agegroups'])

df_14.rename(columns={
    "countryorarea": "country",
    "agegroup": "age_group",
    "maritalstatus": "marital_status",
    "yearstart": "year_start",
    "yearend": "year_end",
    }, inplace =True
    )

df_14.sample(10)

,country,year_start,year_end,sex,marital_status,age_group,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datasource
60482,Denmark,1979,1979,Women,Married or married but separated,[50-54],50,54,78.35,Estimate,1979 Estimate,UNSD
202874,Romania,2003,2003,Men,Single,[20-24],20,24,90.30,Estimate,2003 Estimate,UNSD
151439,Malawi,2004,2005,Men,Divorced,[25-29],25,29,1.80,Survey,2004 DHS,DHS_STATcompiler
244572,Timor-Leste,2015,2015,Men,Married,[40-44],40,44,89.41,Census,2015 Census,National statistics
226565,South Africa,2003,2004,Women,Single,[20-24],20,24,79.40,Survey,2003 DHS,INED
245189,Togo,1988,1988,Women,Divorced,[30-34],30,34,4.60,Survey,1988 DHS,DHS_STATcompiler
134252,Kenya,2008,2009,Women,Divorced,[45-49],45,49,2.80,Survey,2008-2009 DHS,DHS_STATcompiler
213567,Sao Tome and Principe,2000,2000,Women,Divorced,[60-64],60,64,0.00,Survey,2000 MICS_HH,MICS_HH
266405,Viet Nam,2007,2007,Women,Separated,[25-29],25,29,0.59,Survey,2007 Annual APC,National statistics
151649,Malawi,2010,2010,Men,Married,[15-19],15,19,1.90,Survey,2010 DHS,DHS_STATcompiler


In [1606]:
df_14.drop_duplicates(inplace=True)
df_14.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_14.dtypes,
    'null_count': df_14.isnull().sum(),
    'unique_count': df_14.nunique()
})
print(df_info)

                     datatypes  null_count  unique_count
country                 object           0           235
year_start               int64           0            62
year_end                 int64           0            60
sex                     object           0             2
marital_status          object           0            35
age_group               object           0            63
agestart                 int64           0            21
ageend                   int64           0            20
datavalue              float64           0          9994
dataprocess             object           0             6
datacatalogshortname    object           0           443
datasource              object           0            15


In [1607]:
#df_14.to_csv("cleaned_marital_status_by_age_un.csv", index=False)

In [1608]:
#df_14.to_sql('marital_status_by_age_un', engine, if_exists='replace', index=False)

In [1609]:
df_15 = pd.read_csv('../data/processed/regions_un.csv', header=5, low_memory= False)
df_15.head(10)

,Region and subregion,ISO code,Regional Classification,Indicator,Year,AgeGroup,Percentage,Number,DataProcess
0,World,900,M49,Married or in-union women,1970,15-19,22.576683,7.186782e+04,Estimate
1,World,900,M49,Married or in-union women,1970,20-24,63.802057,1.628604e+05,Estimate
2,World,900,M49,Married or in-union women,1970,25-29,87.174827,1.826811e+05,Estimate
3,World,900,M49,Married or in-union women,1970,30-34,90.825027,1.791214e+05,Estimate
4,World,900,M49,Married or in-union women,1970,35-39,90.284386,1.615263e+05,Estimate
5,World,900,M49,Married or in-union women,1970,40-44,86.483531,1.393344e+05,Estimate
6,World,900,M49,Married or in-union women,1970,45-49,82.680237,1.160884e+05,Estimate
7,World,900,M49,Married or in-union women,1970,15-49,69.379111,1.013480e+06,Estimate
8,World,900,M49,Married or in-union women,1971,15-19,22.630416,7.412762e+04,Estimate
9,World,900,M49,Married or in-union women,1971,20-24,63.613178,1.700873e+05,Estimate


In [1610]:
df_15.columns = (df_15.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(','')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
    )
df_15.sample(6)

,regionandsubregion,isocode,regionalclassification,indicator,year,agegroup,percentage,number,dataprocess
2264,Central and Southern Asia,62,SDG,Married or in-union women,2010,15-19,20.917967,37018.104930,Estimate
11728,South-Eastern Asia,35,SDG-M49,Married or in-union women,1978,15-19,19.945482,7403.985559,Estimate
11982,South-Eastern Asia,35,SDG-M49,Married or in-union women,2009,45-49,82.905225,29714.638645,Estimate
22701,Developed countries,901,Development group,Married or in-union women,1972,40-44,83.880085,29083.318654,Estimate
14932,Western Europe,926,M49,Married or in-union women,1973,35-39,84.469603,4583.709655,Estimate
5280,Eastern Africa,910,M49,Married or in-union women,1982,15-19,34.560942,5637.985147,Estimate


In [1611]:
df_15 = df_15.drop(columns=['regionalclassification'])

df_15.rename(columns={
    "regionandsubregion": "region",
    "isocode": "iso_code",
    "agegroup": "age_group",
    "dataprocess": "process"
}, inplace=True)

df_15.sample(10)

,region,iso_code,indicator,year,age_group,percentage,number,process
4748,Africa,903,Married or in-union women,1996,35-39,83.885329,33393.707680,Estimate
25194,Least developed countries,941,Married or in-union women,2041,25-29,73.332935,49589.706220,Projection
18718,Northern America,905,Married or in-union women,2041,45-49,69.774072,8798.008483,Projection
16762,Caribbean,915,Married or in-union women,2040,25-29,42.399112,1351.398341,Projection
17438,Central America,916,Married or in-union women,2043,45-49,63.935546,9400.222135,Projection
13638,Northern Europe,924,Married or in-union women,1973,45-49,83.868229,2277.444691,Estimate
564,World,900,Married or in-union women,2040,35-39,82.054160,456697.750675,Projection
4310,Europe and Northern America,513,Married or in-union women,2022,45-49,70.730919,26332.782570,Estimate
11743,South-Eastern Asia,35,Married or in-union women,1979,15-49,63.610093,104972.554321,Estimate
13142,Eastern Europe,923,Married or in-union women,1992,45-49,77.798057,5743.349750,Estimate


In [1613]:
print(df_15['number'] % 1 != 0)

0        True
1        True
2        True
3        True
4        True
         ... 
28507    True
28508    True
28509    True
28510    True
28511    True
Name: number, Length: 28512, dtype: bool


In [1614]:
df_15['percentage'] = df_15['percentage'].round(2)
df_15['number'] = df_15['number'].astype(int)
df_15.head(10)

,region,iso_code,indicator,year,age_group,percentage,number,process
0,World,900,Married or in-union women,1970,15-19,22.58,71867,Estimate
1,World,900,Married or in-union women,1970,20-24,63.80,162860,Estimate
2,World,900,Married or in-union women,1970,25-29,87.17,182681,Estimate
3,World,900,Married or in-union women,1970,30-34,90.83,179121,Estimate
4,World,900,Married or in-union women,1970,35-39,90.28,161526,Estimate
5,World,900,Married or in-union women,1970,40-44,86.48,139334,Estimate
6,World,900,Married or in-union women,1970,45-49,82.68,116088,Estimate
7,World,900,Married or in-union women,1970,15-49,69.38,1013479,Estimate
8,World,900,Married or in-union women,1971,15-19,22.63,74127,Estimate
9,World,900,Married or in-union women,1971,20-24,63.61,170087,Estimate


In [1751]:
df_15.dropna(inplace=True)
df_15.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_15.dtypes,
    'null_count': df_15.isnull().sum(),
    'unique_count': df_15.nunique()
})
print(df_info)

           datatypes  null_count  unique_count
region        object           0            43
iso_code       int64           0            44
indicator     object           0             1
year           int64           0            81
age_group     object           0             8
percentage   float64           0          7796
number         int32           0         20311
process       object           0             2


In [1615]:
#df_15.to_csv('cleaned_regions_un.csv', index=False)



In [1616]:
#df_15.to_sql('regions_un', engine, if_exists='replace',index=False)

In [1617]:
df_16_1 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa1.csv')
df_16_1
#Data for Chart SF1.1.A. Average size of households by household type, 2024a
# avg_size_all	avg_size_couple_with_children	avg_size_single_parent_with_children		

,Country,All households,Couple households with children,Single parent households with children
0,Mexico,"3,56","4,08","2,76"
1,Costa Rica,"3,46","4,37","3,44"
2,Türkiye,"3,20","4,10","2,80"
3,Israel,"3,19","4,65","2,86"
4,Columbia,"3,10",NaN,NaN
5,Slovak Republic,"3,10","3,80","2,50"
6,Chile,"2,80",NaN,NaN
7,Iceland,"2,70","4,12","2,61"
8,New Zealand,"2,61","3,88","2,67"
9,Greece,"2,60","3,80","2,50"


In [1618]:
df_16_1.columns = df_16_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1619]:
df_16_1.rename(columns={
        "All households": "avg_size_all",
        "Couple with children": "avg_size_couple_with_children",
        "Single parent with children": "avg_size_single_parent_with_children"
}, inplace=True)

In [1620]:
df_16_1.drop_duplicates(inplace=True)
df_16_1.dropna(inplace=True)

In [1621]:
for col in df_16_1.columns:
    if col != 'country':
        # Replace commas with dots if necessary, remove non-numeric chars, convert to float
        df_16_1[col] = (
            df_16_1[col]
            .astype(str)  # ensure string for replace
            .str.replace(',', '.', regex=False)  # decimal commas to dots
            .str.replace(r'[^\d\.\-]', '', regex=True)  # remove non-numeric chars except dot and minus
            .replace('', None)  # empty to NaN
            .astype(float)  # convert to float
        )

# Check updated dtypes
print(df_16_1.dtypes)

country                                    object
all_households                            float64
couple_households_with_children           float64
single_parent_households_with_children    float64
dtype: object


In [1622]:
info_16_1 = pd.DataFrame({
    'dtype': df_16_1.dtypes,
    'null_count': df_16_1.isnull().sum(),
    'unique_count': df_16_1.nunique()
})
print(info_16_1)

                                          dtype  null_count  unique_count
country                                  object           0            39
all_households                          float64           0            19
couple_households_with_children         float64           0            16
single_parent_households_with_children  float64           0            15


In [1623]:
df_16_1.sample(10)

,country,all_households,couple_households_with_children,single_parent_households_with_children
13,Spain,2.50,3.70,2.40
16,Romania,2.50,3.80,2.40
39,Netherlands,2.00,3.90,2.60
28,Switzerland,2.21,4.02,2.58
14,Cyprus,2.50,3.70,2.40
23,Hungary,2.30,3.70,2.40
26,United Kingdom,2.30,3.90,2.80
20,Slovenia,2.40,3.90,2.50
41,Denmark,1.90,3.90,2.50
5,Slovak Republic,3.10,3.80,2.50


In [1624]:
#df_16_1.to_csv('../data/Cleaned/cleaned_average_size_of_households_type_2024_oecd.csv', index=False)

In [1625]:
#df_16_1.to_sql('average_size_of_households_type_2024_oecd', engine, if_exists = 'replace', index= False)

In [1626]:
df_16_2 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa2.csv', header=1)
df_16_2
#Table SF1.1.A. Types of household, 2021a
# share_couple_total	share_couple_with_children	share_couple_without_children	share_single_parent_total	share_single_mother	share_single_father	share_single_person	share_other_types						

,Country,Total,With children,Without children,Total.1,Single mother households,Single father households,Single person households,Other households types
0,Australia,"55,93","29,90","26,02","10,37",NaN,NaN,"25,12","8,58"
1,Austria,"48,93","21,13","27,80","5,63","4,78","0,85","38,34","7,11"
2,Belgium,"52,22","23,98","28,24","7,42","6,08","1,35","35,50","4,86"
3,Canada,"50,92","25,30","25,62","8,72",NaN,NaN,"29,35","11,02"
4,Chile,..,..,..,..,..,..,..,..
5,Columbia,..,..,..,..,..,..,..,..
6,Costa Rica,"52,44","38,15","14,29","10,55","9,49","1,06","11,27","25,74"
7,Czechia,"47,03","21,70","25,32","7,15","6,11","1,04","39,15","6,67"
8,Denmark,"48,60","20,41","28,19","6,31","5,11","1,19","37,57","7,52"
9,Estonia,"46,20","25,46","20,73","6,83","6,09","0,74","36,99","9,98"


In [1627]:
df_16_2.rename(columns={
    "Total": "couple_total(%)",
    "Couple with children": "couple_with_children(%)",
    "Couple without children": "couple_without_children(%)",
    "Total.1": "single_parent_total(%)",
    "Single mother households": "single_mother(%)",
    "Single father households": "single_father(%)",
    "Single person households": "single_person(%)",
    "Other types of households": "other_household_types(%)"
}, inplace=True)

In [1628]:
# Normalize column names (lowercase, underscores, remove () and non-ascii)
df_16_2.columns = (
    df_16_2.columns.str.strip().str.lower()
    .str.replace(' ', '_')
    .str.replace('[()%]', '', regex=True)
    .str.replace('[^0-9a-z_]', '', regex=True)
)

In [1629]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_16_2.columns if c != "country"]

df_16_2[num_cols] = (
    df_16_2[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [1630]:
df_16_2.drop_duplicates(inplace=True)
df_16_2.dropna(inplace=True)
df_16_2.dropna(how="all", subset=num_cols, inplace=True)

In [1631]:
df_16_2.rename(columns={
   "couple_total" : "couple_total(%)",
   "with_children" : "with_children(%)",
   "without_children" : "without_children(%)",
    "single_parent_total" : "single_parent_total(%)",
    "single_mother" : "single_mother(%)",
    "single_father" : "single_father(%)",
    "single_person" : "single_person(%)",
    "other_household_types" : "other_household_types(%)"
}, inplace=True)

In [1632]:
info_16_2 = pd.DataFrame({
    "dtype": df_16_2.dtypes,
    "null_count": df_16_2.isna().sum(),
    "unique_count": df_16_2.nunique()
})
print(info_16_2)
print(df_16_2.dtypes)

                          dtype  null_count  unique_count
country                  object           0            36
couple_total(%)         float64           0            36
with_children(%)        float64           0            35
without_children(%)     float64           0            36
single_parent_total(%)  float64           0            34
single_mother(%)        float64           0            32
single_father(%)        float64           0            31
single_person(%)        float64           0            35
other_households_types  float64           0            36
country                    object
couple_total(%)           float64
with_children(%)          float64
without_children(%)       float64
single_parent_total(%)    float64
single_mother(%)          float64
single_father(%)          float64
single_person(%)          float64
other_households_types    float64
dtype: object


In [1633]:
df_16_2.sample(10)

,country,couple_total(%),with_children(%),without_children(%),single_parent_total(%),single_mother(%),single_father(%),single_person(%),other_households_types
43,Romania,45.66,20.73,24.93,6.50,4.56,1.94,33.63,14.21
30,Slovak Republic,37.15,16.99,20.16,6.23,5.39,0.84,31.40,25.21
22,Lithuania,50.09,23.74,26.35,7.18,6.68,0.50,35.16,7.58
8,Denmark,48.60,20.41,28.19,6.31,5.11,1.19,37.57,7.52
40,Croatia,51.51,24.78,26.73,5.42,4.39,1.04,27.80,15.27
14,Hungary,45.25,20.43,24.82,7.07,5.71,1.37,34.42,13.25
7,Czechia,47.03,21.70,25.32,7.15,6.11,1.04,39.15,6.67
25,Netherlands,53.60,23.01,30.59,6.10,5.00,1.09,38.50,1.80
1,Austria,48.93,21.13,27.80,5.63,4.78,0.85,38.34,7.11
13,Greece,52.14,24.03,28.11,4.66,3.82,0.84,32.35,10.85


In [1634]:
#df_16_2.to_csv('../data/Cleaned/cleaned_types_of_household_2021_oecd.csv', index = False)

In [1635]:
#df_16_2.to_sql('types_of_household_2021_oecd', engine, if_exists = 'replace', index= False)

In [1636]:
df_16_3 = pd.read_csv('../data/Raw/OECD/SF_1_1_Family_size_and_composition - Sayfa3.csv', header=1)
df_16_3
#Table SF1.1.B. Households by number of children, 2024
# share_hh_0_children	share_hh_1_child	share_hh_2_children	share_hh_3plus_children		

,country,0 children,1 child,2 children,3 or more children,Children under 6
0,Australia,..,..,..,..,..
1,Austria,"77,78","10,52","8,57","3,12","9,44"
2,Belgium,"73,97","11,76","10,15","4,11","10,40"
3,Canada,..,..,..,..,..
4,Chile,..,..,..,..,..
5,Columbia,..,..,..,..,..
6,Costa Rica,"30,29","23,08","24,61","22,02","26,30"
7,Czechia,"71,95","13,85","11,56","2,64","12,29"
8,Denmark,"77,78","10,54","8,94","2,74","8,15"
9,Estonia,"75,76","12,53","8,73","2,98","9,85"


In [1637]:
# Normalize column names (lowercase, underscores, remove () and non-ascii)
df_16_3.columns = (
    df_16_3.columns.str.strip().str.lower()
    .str.replace(' ', '_')
    .str.replace('[^0-9a-z_]', '', regex=True)
)

In [1638]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_16_3.columns if c != "country"]

df_16_3[num_cols] = (
    df_16_3[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [1639]:
df_16_3.drop_duplicates(inplace=True)
df_16_3.dropna(inplace=True)

In [1640]:
df_16_3.sample(10)

,country,0_children,1_child,2_children,3_or_more_children,children_under_6
29,Portugal,74.35,15.87,8.18,1.60,9.85
23,Luxembourg,73.00,12.49,12.07,2.41,11.54
28,Poland,74.39,12.91,9.84,2.86,9.97
32,Spain,74.61,13.54,8.95,2.90,8.79
22,Lithuania,80.44,11.06,7.00,1.51,8.12
41,Cyprus,71.36,13.88,11.67,3.10,12.71
35,Türkiye,57.62,17.42,14.50,10.45,19.65
39,Bulgaria,78.21,12.93,7.48,1.38,6.90
2,Belgium,73.97,11.76,10.15,4.11,10.40
9,Estonia,75.76,12.53,8.73,2.98,9.85


In [1641]:
df_16_3.rename(columns={
    "0_children": "households_0_children(%)",
    "1_child": "households_1_child(%)",
    "2_children": "households_2_children(%)",
    "3_or_more_children": "households_3_or_more_children(%)"
}, inplace=True)

In [1642]:
info_16_3 = pd.DataFrame({
    "dtype": df_16_3.dtypes,
    "null_count": df_16_3.isna().sum(),
    "unique_count": df_16_3.nunique()
})
print(info_16_3)
print(df_16_3.dtypes)

                                    dtype  null_count  unique_count
country                            object           0            33
households_0_children(%)          float64           0            32
households_1_child(%)             float64           0            32
households_2_children(%)          float64           0            33
households_3_or_more_children(%)  float64           0            31
children_under_6                  float64           0            31
country                              object
households_0_children(%)            float64
households_1_child(%)               float64
households_2_children(%)            float64
households_3_or_more_children(%)    float64
children_under_6                    float64
dtype: object


In [1643]:
df_16_3.sample(10)

,country,households_0_children(%),households_1_child(%),households_2_children(%),households_3_or_more_children(%),children_under_6
25,Netherlands,78.65,8.78,9.27,3.30,8.79
12,Germany,79.86,9.91,7.72,2.51,8.57
16,Ireland,69.02,12.42,12.18,6.38,11.81
29,Portugal,74.35,15.87,8.18,1.60,9.85
1,Austria,77.78,10.52,8.57,3.12,9.44
6,Costa Rica,30.29,23.08,24.61,22.02,26.30
9,Estonia,75.76,12.53,8.73,2.98,9.85
27,Norway,76.87,10.53,9.14,3.47,8.88
31,Slovenia,75.00,11.25,10.20,3.56,9.93
18,Italy,77.79,12.26,8.28,1.66,8.05


In [1644]:
#df_16_3.to_csv('../data/Cleaned/cleaned_households_by_number_of_children_2024_oecd.csv', index=False)

In [1645]:
#df_16_3.to_sql('households_by_number_of_children_2024_oecd', engine, index= False)

In [1646]:
df_17_1 = pd.read_csv('../data/Raw/OECD/SF_2_1_Total_Fertility_rates_S1.csv')
#total_fertility_rates_from_1960_oecd
df_17_1.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Australia,"3,45","3,55","3,43","3,34","3,15","2,97","2,89","2,85","2,89",...,"1,79","1,79","1,79","1,74","1,74","1,67","1,59","1,70","1,63","1,50"
1,Austria,"2,69","2,78","2,80","2,82","2,79","2,70","2,66","2,62","2,58",...,"1,46","1,49","1,53","1,52","1,48","1,46","1,44","1,48","1,41","1,32"
2,Belgium,"2,54","2,63","2,59","2,68","2,71","2,61","2,52","2,41","2,31",...,"1,74","1,70","1,68","1,65","1,62","1,60","1,55","1,60","1,53","1,47"
3,Canada,"3,90","3,84","3,76","3,67","3,50","3,15","2,81","2,60","2,45",...,"1,61","1,60","1,59","1,55","1,51","1,47","1,41","1,44","1,33","1,26"
4,Chile,"4,70","4,66","4,60","4,54","4,46","4,36","4,26","4,14","4,03",...,"1,77","1,74","1,69","1,56","1,54","1,43","1,31","1,18","1,26","1,17"


In [1647]:
df_info = pd.DataFrame({
    'dtype': df_17_1.dtypes,
    'null_count': df_17_1.isnull().sum(),
    'unique_count': df_17_1.nunique()
})
print(df_info)

          dtype  null_count  unique_count
Country  object           0            49
1960     object           0            47
1961     object           0            47
1962     object           0            47
1963     object           0            46
...         ...         ...           ...
2019     object           0            37
2020     object           0            39
2021     object           0            40
2022     object           0            34
2023     object           0            35

[65 rows x 3 columns]


In [1648]:
df_17_1.columns = df_17_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1649]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_17_1.columns if c != "country"]

df_17_1[num_cols] = (
    df_17_1[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [1650]:
df_17_1.drop_duplicates(inplace=True)
df_17_1.dropna(inplace=True)

In [1651]:
df_info = pd.DataFrame({
    'dtype': df_17_1.dtypes,
    'null_count': df_17_1.isnull().sum(),
    'unique_count': df_17_1.nunique()
})
print(df_info)

           dtype  null_count  unique_count
country   object           0            49
1960     float64           0            47
1961     float64           0            47
1962     float64           0            47
1963     float64           0            46
...          ...         ...           ...
2019     float64           0            37
2020     float64           0            39
2021     float64           0            40
2022     float64           0            34
2023     float64           0            35

[65 rows x 3 columns]


In [1652]:
df_17_1.sample(10)

,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
14,Hungary,2.02,1.94,1.79,1.82,1.80,1.81,1.88,2.01,2.06,...,1.42,1.45,1.51,1.51,1.50,1.51,1.59,1.61,1.55,1.51
25,New Zealand,4.24,4.31,4.19,4.05,3.80,3.54,3.41,3.35,3.34,...,1.92,1.99,1.87,1.81,1.71,1.72,1.61,1.64,1.66,1.56
30,Slovenia,2.18,2.26,2.27,2.28,2.32,2.45,2.48,2.38,2.28,...,1.58,1.57,1.58,1.62,1.60,1.61,1.59,1.64,1.55,1.51
33,Switzerland,2.44,2.53,2.60,2.67,2.68,2.61,2.52,2.41,2.30,...,1.54,1.54,1.54,1.52,1.52,1.48,1.46,1.52,1.39,1.33
20,Latvia,1.94,1.94,1.91,1.85,1.79,1.74,1.76,1.80,1.83,...,1.65,1.70,1.74,1.69,1.60,1.61,1.55,1.57,1.47,1.36
31,Spain,2.86,2.76,2.80,2.88,3.01,2.94,2.99,3.03,2.96,...,1.32,1.33,1.34,1.31,1.26,1.23,1.19,1.19,1.16,1.12
32,Sweden,2.20,2.23,2.26,2.34,2.48,2.42,2.36,2.27,2.07,...,1.88,1.85,1.85,1.78,1.75,1.70,1.66,1.67,1.52,1.45
17,Italy,2.41,2.41,2.46,2.56,2.70,2.67,2.63,2.54,2.49,...,1.38,1.36,1.36,1.34,1.31,1.27,1.24,1.25,1.24,1.20
23,Mexico,6.77,6.76,6.76,6.75,6.75,6.76,6.77,6.79,6.81,...,2.21,2.14,2.09,2.04,2.03,2.02,1.99,1.97,1.94,1.91
44,Argentina,3.14,3.12,3.13,3.11,3.08,3.01,2.97,2.98,3.01,...,2.39,2.35,2.24,2.17,2.07,1.88,1.60,1.59,1.48,1.50


In [1653]:
#df_17_1.to_csv('../data/Cleaned/cleaned_total_fertility_rates_oecd.csv', index=False)

In [1654]:
#df_17_1.to_sql('total_fertility_rates_oecd', engine, if_exists='replace', index=False)

In [1655]:
df_17_2 = pd.read_csv('../data/Raw/OECD/SF_2_1_Fertility_rates_Births_by_birth_order_S2.csv')
df_17_2

,Country,Birth order,1987,1988,1989,1990,1991,1992,1993,1994,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Austria,First birth,"47,6","47,8","46,7","46,2","46,5","46,1","45,2","44,5",...,"48,0","48,3","47,3","47,5","47,1","47,2","47,7","47,6","48,4","48,1"
1,Austria,Second birth,"33,7","33,7","34,3","34,9","34,5","34,8","35,8","36,4",...,"35,5","35,3","35,6","35,3","35,3","35,1","35,3","35,5","34,9","35,1"
2,Austria,Third birth or higher,"18,8","18,5","19,0","18,9","19,0","19,1","18,9","19,1",...,"16,5","16,4","17,1","17,2","17,6","17,7","17,0","16,9","16,7","16,8"
3,Belgium,First birth,"46,8","46,9","47,3","47,3","48,1","47,2","46,9","47,2",...,"42,3","43,5","44,1","43,6","42,9","42,6","45,0","44,0","44,7","45,5"
4,Belgium,Second birth,"33,0","32,9","32,7","32,8","32,3","32,8","33,5","33,0",...,"35,1","34,8","34,5","34,6","34,5","34,7","34,2","35,1","34,3","34,1"
5,Belgium,Third birth or higher,"20,2","20,2","19,9","19,9","19,6","20,0","19,6","19,8",...,"22,6","21,8","21,4","21,9","22,6","22,6","20,8","20,9","20,9","20,4"
6,Czechia,First birth,"46,7","46,6","47,4","47,8","50,1","49,8","48,5","47,7",...,"47,4","48,1","48,7","48,7","48,0","47,8","47,6","46,4","46,3","46,3"
7,Czechia,Second birth,"37,7","37,6","37,4","37,2","35,5","35,8","36,8","36,9",...,"37,5","37,3","36,7","36,6","37,2","37,6","37,6","39,0","38,6","39,1"
8,Czechia,Third birth or higher,"15,6","15,8","15,2","15,0","14,4","14,4","14,8","15,4",...,"15,1","14,7","14,6","14,7","14,7","14,6","14,8","14,6",15,"14,6"
9,Estonia,First birth,"43,5","43,5","44,0","46,2","49,5","50,3","49,6","49,6",...,"41,9","42,3","40,8","40,2","36,7","38,8","38,0","37,2","39,8","39,7"


In [1656]:
df_info = pd.DataFrame({
    'dtype': df_17_2.dtypes,
    'null_count': df_17_2.isnull().sum(),
    'unique_count': df_17_2.nunique()
})
print(df_info)

              dtype  null_count  unique_count
Country      object           0            17
Birth order  object           0             3
1987         object           0            48
1988         object           0            49
1989         object           0            48
1990         object           0            44
1991         object           0            48
1992         object           0            46
1993         object           0            47
1994         object           0            47
1995         object           0            48
1996         object           0            47
1997         object           0            49
1998         object           0            50
1999         object           0            49
2000         object           0            48
2001         object           0            50
2002         object           0            47
2003         object           0            50
2004         object           0            49
2005         object           0   

In [1657]:
df_17_2.columns = df_17_2.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_17_2.head()

,country,birth_order,1987,1988,1989,1990,1991,1992,1993,1994,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Austria,First birth,"47,6","47,8","46,7","46,2","46,5","46,1","45,2","44,5",...,"48,0","48,3","47,3","47,5","47,1","47,2","47,7","47,6","48,4","48,1"
1,Austria,Second birth,"33,7","33,7","34,3","34,9","34,5","34,8","35,8","36,4",...,"35,5","35,3","35,6","35,3","35,3","35,1","35,3","35,5","34,9","35,1"
2,Austria,Third birth or higher,"18,8","18,5","19,0","18,9","19,0","19,1","18,9","19,1",...,"16,5","16,4","17,1","17,2","17,6","17,7","17,0","16,9","16,7","16,8"
3,Belgium,First birth,"46,8","46,9","47,3","47,3","48,1","47,2","46,9","47,2",...,"42,3","43,5","44,1","43,6","42,9","42,6","45,0","44,0","44,7","45,5"
4,Belgium,Second birth,"33,0","32,9","32,7","32,8","32,3","32,8","33,5","33,0",...,"35,1","34,8","34,5","34,6","34,5","34,7","34,2","35,1","34,3","34,1"


In [1658]:
# --- Ensure "country" and "birth order" are strings (tidy casing/spacing) ---
df_17_2["country"] = df_17_2["country"].astype(str).str.strip().str.title()
df_17_2["birth_order"] = df_17_2["birth_order"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_17_2.columns if c not in ["country", "birth_order"]]
# --- Robust cleaning -> convert to float ---
df_17_2[num_cols] = (
    df_17_2[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_17_2[num_cols] = df_17_2[num_cols].round(2)


In [1659]:
df_17_2.drop_duplicates(inplace=True)
df_17_2.dropna(inplace=True)

In [1660]:
df_17_2["birth_order"] = df_17_2["birth_order"].astype(str) + "(%)"

In [1661]:
df_17_2.head(10)

,country,birth_order,1987,1988,1989,1990,1991,1992,1993,1994,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Austria,First birth(%),47.6,47.8,46.7,46.2,46.5,46.1,45.2,44.5,...,48.0,48.3,47.3,47.5,47.1,47.2,47.7,47.6,48.4,48.1
1,Austria,Second birth(%),33.7,33.7,34.3,34.9,34.5,34.8,35.8,36.4,...,35.5,35.3,35.6,35.3,35.3,35.1,35.3,35.5,34.9,35.1
2,Austria,Third birth or higher(%),18.8,18.5,19.0,18.9,19.0,19.1,18.9,19.1,...,16.5,16.4,17.1,17.2,17.6,17.7,17.0,16.9,16.7,16.8
3,Belgium,First birth(%),46.8,46.9,47.3,47.3,48.1,47.2,46.9,47.2,...,42.3,43.5,44.1,43.6,42.9,42.6,45.0,44.0,44.7,45.5
4,Belgium,Second birth(%),33.0,32.9,32.7,32.8,32.3,32.8,33.5,33.0,...,35.1,34.8,34.5,34.6,34.5,34.7,34.2,35.1,34.3,34.1
5,Belgium,Third birth or higher(%),20.2,20.2,19.9,19.9,19.6,20.0,19.6,19.8,...,22.6,21.8,21.4,21.9,22.6,22.6,20.8,20.9,20.9,20.4
6,Czechia,First birth(%),46.7,46.6,47.4,47.8,50.1,49.8,48.5,47.7,...,47.4,48.1,48.7,48.7,48.0,47.8,47.6,46.4,46.3,46.3
7,Czechia,Second birth(%),37.7,37.6,37.4,37.2,35.5,35.8,36.8,36.9,...,37.5,37.3,36.7,36.6,37.2,37.6,37.6,39.0,38.6,39.1
8,Czechia,Third birth or higher(%),15.6,15.8,15.2,15.0,14.4,14.4,14.8,15.4,...,15.1,14.7,14.6,14.7,14.7,14.6,14.8,14.6,15.0,14.6
9,Estonia,First birth(%),43.5,43.5,44.0,46.2,49.5,50.3,49.6,49.6,...,41.9,42.3,40.8,40.2,36.7,38.8,38.0,37.2,39.8,39.7


In [1662]:
df_info = pd.DataFrame({
    'dtype': df_17_2.dtypes,
    'null_count': df_17_2.isnull().sum(),
    'unique_count': df_17_2.nunique()
})
print(df_info)

               dtype  null_count  unique_count
country       object           0            17
birth_order   object           0             3
1987         float64           0            48
1988         float64           0            49
1989         float64           0            48
1990         float64           0            44
1991         float64           0            48
1992         float64           0            46
1993         float64           0            47
1994         float64           0            47
1995         float64           0            48
1996         float64           0            47
1997         float64           0            49
1998         float64           0            50
1999         float64           0            49
2000         float64           0            48
2001         float64           0            50
2002         float64           0            47
2003         float64           0            50
2004         float64           0            49
2005         

In [1663]:
#df_17_2.to_csv('../data/Cleaned/cleaned_births_by_birth_order_oecd.csv', index=False)

In [1664]:
#df_17_2.to_sql('births_by_birth_order_oecd', engine, if_exists='replace', index=False)

In [1665]:
df_18 = pd.read_csv('../data/Raw/OECD/sf1_2_wide_from_df18.csv')
df_18

,country,year,Living with two parents,Living with a single parent,Other
0,Australia,2003,80.1,19.5,0.5
1,Australia,2006,81.5,18.0,0.5
2,Australia,2009,82.0,17.6,0.4
3,Australia,2012,81.3,18.0,0.6
4,Austria,2003,81.2,16.8,2.0
...,...,...,...,...,...
470,United States,2014,68.7,27.5,3.8
471,United States,2015,69.2,26.8,3.9
472,United States,2016,68.7,27.4,3.8
473,United States,2017,68.9,27.1,4.0


In [1666]:
for col in df_18.select_dtypes(include=['object']).columns:
    df_18[col] = df_18[col].astype(str).str.strip()

# 2) Define placeholders representing missing data in OECD exports
placeholders = ['..', '...', '.', ' .', '…', 'Na', 'nan', 'None']

# 3) Replace placeholders with NaN directly in df_18
df_18.replace(placeholders, pd.NA, inplace=True)

In [1667]:
# 1) Ensure 'year' is integer
df_18["year"] = pd.to_numeric(df_18["year"], errors="coerce").astype("Int64")

# 2) Convert all non-key columns to numeric and round(2)
for col in df_18.columns:
    if col not in ["country", "year"]:
        df_18[col] = pd.to_numeric(df_18[col], errors="coerce").round(2)

In [1668]:
# 1) Drop rows with missing key fields
df_18.dropna(subset=["country", "year"], inplace=True)

# 2) Drop duplicate country-year rows, keep the first
df_18.drop_duplicates(subset=["country", "year"], keep="first", inplace=True)

# 3) Drop rows where all value columns are NaN
value_cols = [c for c in df_18.columns if c not in ["country", "year"]]
df_18.dropna(subset=value_cols, how="all", inplace=True)

# 4) Sort and reset index
df_18.sort_values(["country", "year"], inplace=True)
df_18.reset_index(drop=True, inplace=True)


In [1669]:
df_18

,country,year,Living with two parents,Living with a single parent,Other
0,Australia,2003,80.1,19.5,0.5
1,Australia,2006,81.5,18.0,0.5
2,Australia,2009,82.0,17.6,0.4
3,Australia,2012,81.3,18.0,0.6
4,Austria,2003,81.2,16.8,2.0
...,...,...,...,...,...
470,United States,2014,68.7,27.5,3.8
471,United States,2015,69.2,26.8,3.9
472,United States,2016,68.7,27.4,3.8
473,United States,2017,68.9,27.1,4.0


In [1670]:
df_18['Other'] = pd.to_numeric(df_18['Other'], errors='coerce')

In [1671]:
df_info = pd.DataFrame({
    'dtype': df_18.dtypes,
    'null_count': df_18.isnull().sum(),
    'unique_count': df_18.nunique()
})
print(df_info)

                               dtype  null_count  unique_count
country                       object           0            39
year                           Int64           0            18
Living with two parents      float64           0           211
Living with a single parent  float64           0           203
Other                        float64           1            50


In [1672]:
print(repr(df_18.loc[df_18['Other'].notnull(), 'Other'].unique()))

array([0.5, 0.4, 0.6, 2. , 1. , 1.9, 0.3, 0.1, 0.8, 0.7, 8.7, 3.5, 2.5,
       2.1, 2.4, 2.6, 6.7, 5.1, 1.4, 1.2, 1.7, 1.5, 3.4, 2.9, 2.3, 3. ,
       4.2, 2.8, 1.3, 9. , 0.2, 0.9, 1.1, 4.5, 4.7, 1.6, 3.8, 3.6, 3.3,
       2.2, 0. , 1.8, 2.7, 3.2, 3.9, 4.1, 4.4, 3.7, 4. , 4.3])


In [1673]:
df_18['Other'] = pd.to_numeric(df_18['Other'], errors='coerce')

df_18.dropna(inplace=True, subset=['Other'])

df_18.isnull().sum()

country                        0
year                           0
Living with two parents        0
Living with a single parent    0
Other                          0
dtype: int64

In [1674]:
#df_18.to_csv('../data/Cleaned/cleaned_household_children.csv', index=False)

In [1675]:
#df_18.to_sql('household_children_oecd', engine, if_exists= 'replace', index= False)

In [1676]:
df_19_1 =pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_mean_age_birth_S1.csv')
#age_of_mothers_at_childbirth
df_19_1

,Country,1963,1964,1965,1966,1967,1968,1969,1970,1971,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,"27,5","27,5","27,4","27,3","27,3","27,2","27,2","27,1","26,9",...,"30,1","30,1","30,2","30,3","30,5","30,6","30,7","30,8","30,9","31,1"
1,Austria,"27,4","27,4","27,3","27,1","27,0","26,8","26,8","26,7","26,7",...,"30,2","30,3","30,4","30,6","30,6","30,7","30,9","31,0","31,0","31,2"
2,Belgium,"27,8","27,7","27,6","27,5","27,4","27,3","27,2","27,2","27,0",...,"30,0","30,2","30,3","30,4","30,5","30,6","30,7","30,8","30,8","31,0"
3,Canada,"27,8","27,9","27,8","27,7","27,5","27,3","27,3","27,2","27,0",...,"30,3","30,4","30,5","30,6","30,7","30,9","31,0","31,2","31,3","31,4"
4,Chile,"29,2","29,1","29,1","29,0","28,8","28,7","28,6","28,4","28,2",...,"28,1","28,3","28,5","28,8","29,1","29,4","29,6","29,9","30,1",..
5,Czech Republic,"25,7","25,8","25,5","25,2","25,0","24,9","24,8","24,8","24,9",...,"29,8","29,9","29,9","30,0","30,0","30,0","30,1","30,2","30,2","30,4"
6,Costa Rica,"29,3","29,3","29,3","29,3","29,2","29,1","28,9","28,7","28,5",...,"26,5","26,7","26,8","27,1","27,2","27,4","27,6","27,9","28,4","28,7"
7,Denmark,"27,3","26,8","26,8","26,6","26,5","26,5","26,6","26,7","26,7",...,"30,7","30,8","30,9","31,0","31,0","31,1","31,2","31,3","31,4","31,6"
8,Estonia,"27,6","27,4","27,3","27,3","27,1","26,9","26,9","26,7","26,7",...,"29,6","29,5","29,6","29,9","30,2","30,4","30,5","30,6","30,7","31,0"
9,Finland,"28,1","28,0","28,0","27,8","27,7","27,5","27,4","27,1","26,9",...,"30,4","30,5","30,5","30,6","30,8","30,9","31,0","31,1","31,2","31,4"


In [1677]:
df_info = pd.DataFrame({
    'dtype': df_19_1.dtypes,
    'null_count': df_19_1.isnull().sum(),
    'unique_count': df_19_1.nunique()
})
print(df_info)

          dtype  null_count  unique_count
Country  object           0            26
1963     object           0            19
1964     object           0            22
1965     object           0            22
1966     object           0            22
1967     object           0            22
1968     object           0            20
1969     object           0            21
1970     object           0            19
1971     object           0            19
1972     object           0            20
1973     object           0            20
1974     object           0            24
1975     object           0            21
1976     object           0            22
1977     object           0            20
1978     object           0            22
1979     object           0            23
1980     object           0            22
1981     object           0            20
1982     object           0            18
1983     object           0            20
1984     object           0       

In [1678]:
df_19_1.columns = df_19_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1679]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_19_1.columns if c != "country"]

df_19_1[num_cols] = (
    df_19_1[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [1680]:
df_19_1.drop_duplicates(inplace=True)
df_19_1.dropna(inplace=True)

In [1681]:
df_info = pd.DataFrame({
    'dtype': df_19_1.dtypes,
    'null_count': df_19_1.isnull().sum(),
    'unique_count': df_19_1.nunique()
})
print(df_info)

           dtype  null_count  unique_count
country   object           0            22
1963     float64           0            16
1964     float64           0            18
1965     float64           0            18
1966     float64           0            18
1967     float64           0            18
1968     float64           0            17
1969     float64           0            17
1970     float64           0            15
1971     float64           0            17
1972     float64           0            18
1973     float64           0            18
1974     float64           0            20
1975     float64           0            18
1976     float64           0            18
1977     float64           0            16
1978     float64           0            18
1979     float64           0            21
1980     float64           0            20
1981     float64           0            17
1982     float64           0            17
1983     float64           0            18
1984     fl

In [1682]:
df_19_1.sample(10)

,country,1963,1964,1965,1966,1967,1968,1969,1970,1971,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
9,Finland,28.1,28.0,28.0,27.8,27.7,27.5,27.4,27.1,26.9,...,30.4,30.5,30.5,30.6,30.8,30.9,31.0,31.1,31.2,31.4
23,United States,26.5,26.6,26.6,26.4,26.3,26.3,26.2,26.1,26.0,...,28.0,28.2,28.3,28.5,28.7,28.8,29.0,29.1,29.2,29.4
0,Australia,27.5,27.5,27.4,27.3,27.3,27.2,27.2,27.1,26.9,...,30.1,30.1,30.2,30.3,30.5,30.6,30.7,30.8,30.9,31.1
2,Belgium,27.8,27.7,27.6,27.5,27.4,27.3,27.2,27.2,27.0,...,30.0,30.2,30.3,30.4,30.5,30.6,30.7,30.8,30.8,31.0
13,Ireland,31.1,31.1,31.0,30.9,30.8,30.6,30.5,30.4,30.2,...,31.5,31.7,31.8,31.9,32.1,32.1,32.2,32.4,32.6,32.7
22,Switzerland,27.8,27.8,27.7,27.7,27.5,27.5,27.9,27.8,27.7,...,31.5,31.6,31.8,31.8,31.9,31.9,32.0,32.1,32.2,32.3
17,Netherlands,29.3,29.2,29.0,28.8,28.5,28.4,28.3,28.2,28.0,...,30.9,31.0,31.1,31.2,31.3,31.4,31.5,31.6,31.7,31.8
8,Estonia,27.6,27.4,27.3,27.3,27.1,26.9,26.9,26.7,26.7,...,29.6,29.5,29.6,29.9,30.2,30.4,30.5,30.6,30.7,31.0
18,New Zealand,27.5,27.4,27.2,27.0,26.9,26.8,26.8,26.7,26.5,...,29.8,29.9,30.1,30.2,30.4,30.4,30.6,30.7,30.8,31.0
19,Norway,27.8,27.8,27.7,27.6,27.5,27.3,27.2,27.0,26.8,...,30.3,30.5,30.6,30.7,30.8,31.0,31.1,31.3,31.4,31.5


In [1683]:
#df_19_1.to_csv('../data/Cleaned/age_of_mothers_at_childbirth_oecd.csv', index=False)

In [1684]:
#df_19_1.to_sql('age_of_mothers_at_childbirth_oecd', engine, if_exists='replace', index=False)

In [1685]:
df_19_2 = pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_fertility_by_age_1960_S2.csv')
#fertility_per_1000_from 1960
df_19_2.head()

,Country,Age group,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,15-19,"44,3","47,4","44,7","45,9","47,0","47,5","48,9","48,4",...,"16,1","14,6","12,9","12,0","10,5","10,3","9,5","8,8","7,9","7,1"
1,Australia,20-24,"220,1","225,8","216,0","208,2","190,5","179,3","173,1","170,8",...,"53,2","51,3","47,4","47,3","44,7","43,1","42,8","40,1","37,7","38,8"
2,Australia,25-29,"216,3","221,2","216,7","211,2","198,1","188,5","183,9","185,0",...,"102,6","99,1","94,8","93,4","92,2","89,7","89,3","84,3","80,3","86,7"
3,Australia,30-34,"127,5","131,1","127,7","123,9","119,1","110,1","105,1","102,8",...,"126,9","124,8","120,4","121,7","123,6","119,1","120,1","115,6","111,4","120,6"
4,Australia,35-39,"62,3","63,4","61,4","59,7","58,4","53,0","50,6","47,8",...,"71,5","70,9","69,2","69,8","72,0","71,3","71,6","69,3","66,3","70,9"


In [1686]:
df_info = pd.DataFrame({
    'dtype': df_19_2.dtypes,
    'null_count': df_19_2.isnull().sum(),
    'unique_count': df_19_2.nunique()
})
print(df_info)

            dtype  null_count  unique_count
Country    object           0            21
Age group  object           0             7
1960       object           0           136
1961       object           0           140
1962       object           0           140
...           ...         ...           ...
2017       object           0           124
2018       object           0           128
2019       object           0           126
2020       object           0           121
2021       object           7           119

[64 rows x 3 columns]


In [1687]:
df_19_2.columns = df_19_2.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_19_2.head()

,country,age_group,1960,1961,1962,1963,1964,1965,1966,1967,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Australia,15-19,"44,3","47,4","44,7","45,9","47,0","47,5","48,9","48,4",...,"16,1","14,6","12,9","12,0","10,5","10,3","9,5","8,8","7,9","7,1"
1,Australia,20-24,"220,1","225,8","216,0","208,2","190,5","179,3","173,1","170,8",...,"53,2","51,3","47,4","47,3","44,7","43,1","42,8","40,1","37,7","38,8"
2,Australia,25-29,"216,3","221,2","216,7","211,2","198,1","188,5","183,9","185,0",...,"102,6","99,1","94,8","93,4","92,2","89,7","89,3","84,3","80,3","86,7"
3,Australia,30-34,"127,5","131,1","127,7","123,9","119,1","110,1","105,1","102,8",...,"126,9","124,8","120,4","121,7","123,6","119,1","120,1","115,6","111,4","120,6"
4,Australia,35-39,"62,3","63,4","61,4","59,7","58,4","53,0","50,6","47,8",...,"71,5","70,9","69,2","69,8","72,0","71,3","71,6","69,3","66,3","70,9"


In [1688]:
# --- Ensure "country" and "age_group" are strings
df_19_2["country"] = df_19_2["country"].astype(str).str.strip().str.title()
df_19_2["age_group"] = df_19_2["age_group"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_19_2.columns if c not in ["country", "age_group"]]
# --- Robust cleaning -> convert to float ---
df_19_2[num_cols] = (
    df_19_2[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_19_2[num_cols] = df_19_2[num_cols].round(2)

In [1689]:
df_19_2.drop_duplicates(inplace=True)
df_19_2.dropna(inplace = True)

In [1690]:
df_info = pd.DataFrame({
    'dtype': df_19_2.dtypes,
    'null_count': df_19_2.isnull().sum(),
    'unique_count': df_19_2.nunique()
})
print(df_info)

             dtype  null_count  unique_count
country     object           0            19
age_group   object           0             7
1960       float64           0           124
1961       float64           0           126
1962       float64           0           126
...            ...         ...           ...
2017       float64           0           118
2018       float64           0           121
2019       float64           0           120
2020       float64           0           115
2021       float64           0           118

[64 rows x 3 columns]


In [1691]:
#df_19_2.to_csv('../data/Cleaned/fertility_per_1000_by_age_from 1960_oecd.csv', index=False)

In [1692]:
#df_19_2.to_sql('fertility_per_1000_from_1960_oecd', engine, if_exists='replace', index=False)

In [1693]:
df_19_3 = pd.read_csv('../data/Raw/OECD/SF_2_3_Age_mothers_childbirth_fertility_by_age_2000_S3.csv')
#fertility_per_1000_from_2000
df_19_3

,Country,Age group,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,OECD-Average,15-19,"22,6","22,0","21,1","20,5","20,3","20,1","20,0","20,5",...,"17,9","16,8","16,2","15,2","14,4","13,5","12,6","11,7","10,2","9,5"
1,OECD-Average,20-24,"71,7","69,3","66,8","65,5","64,7","63,2","62,9","63,0",...,"56,4","53,8","53,3","51,9","50,4","48,8","47,0","45,0","42,0","40,5"
2,OECD-Average,25-29,"107,9","105,0","103,1","103,5","103,4","102,3","102,6","103,4",...,"99,4","96,5","96,9","96,1","94,9","92,8","90,7","88,4","85,5","86,9"
3,OECD-Average,30-34,"88,1","87,2","88,6","91,1","93,4","94,6","97,6","100,0",...,"103,6","101,9","104,0","104,9","105,3","104,1","103,3","101,7","99,6","103,6"
4,OECD-Average,35-39,"38,1","38,6","39,5","40,6","42,2","43,5","45,6","47,7",...,"53,1","53,4","55,1","56,3","57,1","57,0","57,4","57,5","55,9","58,7"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,Romania,25-29,"78,2","77,0","78,6","82,0","84,8","90,8","92,3","93,0",...,"91,8","88,3","94,4","98,9","100,1","109,0","108,3","109,1","109,4","110,9"
297,Romania,30-34,"38,8","38,1","38,8","38,8","41,6","47,5","51,1","54,2",...,"66,6","64,8","71,5","75,4","78,5","86,6","85,9","86,4","87,1","87,5"
298,Romania,35-39,"13,4","13,8","15,2","19,4","23,2","25,1","25,7","24,9",...,"27,3","27,4","29,9","32,1","33,0","36,8","36,7","38,3","40,6","41,1"
299,Romania,40-44,"3,1","3,1","3,0","2,9","3,1","3,1","2,8","3,1",...,"4,9","4,8","5,6","6,1","6,8","7,3","7,8","8,0","8,5","8,2"


In [1694]:
df_info = pd.DataFrame({
    'dtype': df_19_3.dtypes,
    'null_count': df_19_3.isnull().sum(),
    'unique_count': df_19_3.nunique()
})
print(df_info)

            dtype  null_count  unique_count
Country    object           0            43
Age group  object           0             7
2000       object           0           233
2001       object           0           248
2002       object           0           240
2003       object           0           239
2004       object           0           245
2005       object           0           240
2006       object           0           239
2007       object           0           242
2008       object           0           252
2009       object           0           251
2010       object           0           239
2011       object           0           235
2012       object           0           242
2013       object           0           234
2014       object           0           238
2015       object           0           237
2016       object           0           248
2017       object           0           236
2018       object           0           245
2019       object           0   

In [1695]:
df_19_3.columns = df_19_3.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_19_3.head()

,country,age_group,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,OECD-Average,15-19,"22,6","22,0","21,1","20,5","20,3","20,1","20,0","20,5",...,"17,9","16,8","16,2","15,2","14,4","13,5","12,6","11,7","10,2","9,5"
1,OECD-Average,20-24,"71,7","69,3","66,8","65,5","64,7","63,2","62,9","63,0",...,"56,4","53,8","53,3","51,9","50,4","48,8","47,0","45,0","42,0","40,5"
2,OECD-Average,25-29,"107,9","105,0","103,1","103,5","103,4","102,3","102,6","103,4",...,"99,4","96,5","96,9","96,1","94,9","92,8","90,7","88,4","85,5","86,9"
3,OECD-Average,30-34,"88,1","87,2","88,6","91,1","93,4","94,6","97,6","100,0",...,"103,6","101,9","104,0","104,9","105,3","104,1","103,3","101,7","99,6","103,6"
4,OECD-Average,35-39,"38,1","38,6","39,5","40,6","42,2","43,5","45,6","47,7",...,"53,1","53,4","55,1","56,3","57,1","57,0","57,4","57,5","55,9","58,7"


In [1696]:
# --- Ensure "country" and "age_group" are strings
df_19_3["country"] = df_19_3["country"].astype(str).str.strip().str.title()
df_19_3["age_group"] = df_19_3["age_group"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_19_3.columns if c not in ["country", "age_group"]]
# --- Robust cleaning -> convert to float ---
df_19_3[num_cols] = (
    df_19_3[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_19_3[num_cols] = df_19_3[num_cols].round(2)

In [1697]:
df_19_3.drop_duplicates(inplace=True)
df_19_3.dropna(inplace=True)

In [1698]:
#Check again
df_info = pd.DataFrame({
    'dtype': df_19_3.dtypes,
    'null_count': df_19_3.isnull().sum(),
    'unique_count': df_19_3.nunique()
})
print(df_info)

             dtype  null_count  unique_count
country     object           0            41
age_group   object           0             7
2000       float64           0           225
2001       float64           0           237
2002       float64           0           232
2003       float64           0           229
2004       float64           0           233
2005       float64           0           229
2006       float64           0           229
2007       float64           0           230
2008       float64           0           238
2009       float64           0           238
2010       float64           0           230
2011       float64           0           227
2012       float64           0           231
2013       float64           0           225
2014       float64           0           226
2015       float64           0           225
2016       float64           0           237
2017       float64           0           227
2018       float64           0           233
2019      

In [1699]:
df_19_3.sample(10)

,country,age_group,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
57,Czech Republic,20-24,67.8,61.5,58.5,54.0,51.0,49.1,47.3,48.7,...,42.4,41.9,43.0,45.5,49.4,50.8,51.3,49.7,49.7,47.6
185,Netherlands,30-34,130.2,129.0,131.2,131.3,129.4,127.6,128.8,128.1,...,130.6,127.9,132.1,129.0,129.8,127.8,126.3,125.6,124.9,133.5
276,Bulgaria,30-34,32.3,32.5,34.3,37.7,42.5,48.4,53.6,58.0,...,65.8,64.6,68.5,69.7,71.2,71.5,72.3,72.8,71.3,73.6
92,Germany,20-24,56.9,54.6,52.4,50.5,49.5,47.3,45.3,43.9,...,37.1,35.9,35.7,35.8,39.3,36.7,35.8,34.0,32.5,31.2
73,Estonia,30-34,52.7,51.9,56.7,57.9,64.5,69.8,74.1,81.2,...,85.8,82.4,84.7,89.1,95.4,95.2,103.5,101.3,98.8,102.6
5,Oecd-Average,40-44,7.6,7.8,8.0,8.3,8.6,8.8,9.1,9.6,...,11.3,11.5,11.8,12.2,12.8,12.9,13.1,13.4,13.3,13.7
202,Norway,45-49,0.2,0.3,0.2,0.3,0.3,0.4,0.4,0.4,...,0.6,0.5,0.7,0.8,0.7,0.7,0.8,0.9,0.7,0.8
300,Romania,45-49,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,...,0.3,0.3,0.3,0.3,0.3,0.4,0.4,0.5,0.6,0.5
100,Greece,25-29,81.2,83.1,84.9,84.0,84.0,82.7,83.5,82.2,...,77.2,72.2,70.1,68.7,68.8,65.7,64.4,61.9,64.6,64.1
220,Slovak Republic,30-34,44.2,43.5,46.1,50.5,55.0,59.2,62.9,66.4,...,76.1,76.0,78.6,80.9,84.1,86.4,87.5,87.8,89.0,92.2


In [1700]:
#df_19_3.to_csv('../data/Cleaned/cleaned_fertility_per_1000_from_2000_oecd.csv',index=False)

In [1701]:
#df_19_3.to_sql('fertility_per_1000_from_2000_oecd',engine, if_exists='replace', index=False)

In [1702]:
df_20= pd.read_csv('../data/Raw/OECD/SF_2_4_Share_births_outside_marriage_1960.csv')
#(%)share_of_births_outside_of_marriage
df_20

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Austria,"13,0","12,6","12,0","11,6","11,3","11,2","11,4","11,5","12,0",...,"40,4","41,5","41,4","41,7","42,1","42,2","42,0","41,3","40,6","41,2"
1,Belgium,"2,1","2,0","2,1","2,2","2,3","2,4","2,5","2,5","2,7",...,"47,0","47,7","49,5","49,4","48,0","49,0","52,8","52,4",..,..
2,Czech Republic,"4,9","4,6","4,5","4,7","4,8","5,0","5,3","5,3","5,4",...,"41,8","43,4","45,0","46,7","47,8","48,6","49,0","48,5","48,2","48,5"
3,Denmark,"7,8","8,0","8,3","8,9","9,3","9,5","10,2","11,1","11,1",...,"49,0","50,6","51,5","52,5","53,8","54,0","54,2","54,2","54,1","54,2"
4,Finland,"4,0","4,1","4,0","4,2","4,4","4,6","4,8","5,1","5,3",...,"40,9","41,5","42,1","42,8","44,3","44,9","44,8","44,6","45,4","46,1"
5,Germany,"7,6","7,1","6,6","6,1","5,9","5,8","5,7","5,8","6,1",...,"33,9","34,5","34,8","35,0","35,0","35,5","34,7","33,9","33,3","33,1"
6,Greece,"1,2","1,2","1,2","1,2","1,1","1,1","1,0","1,0","1,1",...,"7,4","7,6","7,0","8,2","8,8","9,4","10,3","11,1","12,4","13,8"
7,Hungary,"5,5","5,5","5,4","5,3","5,2","5,2","5,1","5,0","5,0",...,"42,3","44,5","45,6","47,3","47,9","46,7","44,7","43,9","38,7","30,4"
8,Iceland,"25,3","25,3","24,5","25,1","26,7","26,9","28,4","30,0","30,5",...,"65,0","66,9",..,..,..,"69,6","71,2","70,5","69,4",..
9,Ireland,"1,6","1,6","1,8","1,8","2,0","2,2","2,3","2,5","2,6",...,"33,9","35,1","35,3","36,3","36,6","36,7","37,6","37,9","38,4",..


In [1703]:
df_info = pd.DataFrame({
    'dtype': df_20.dtypes,
    'null_count': df_20.isnull().sum(),
    'unique_count': df_20.nunique()
})
print(df_info)

          dtype  null_count  unique_count
Country  object           0            26
1960     object           0            26
1961     object           0            24
1962     object           0            24
1963     object           0            24
...         ...         ...           ...
2016     object           0            24
2017     object           0            26
2018     object           0            25
2019     object           0            25
2020     object           0            24

[62 rows x 3 columns]


In [1704]:
df_20.columns = df_20.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1705]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_20.columns if c != "country"]

df_20[num_cols] = (
    df_20[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [1706]:
df_20.drop_duplicates(inplace=True)
df_20.dropna(inplace=True)

df_info = pd.DataFrame({
    'dtype': df_20.dtypes,
    'null_count': df_20.isnull().sum(),
    'unique_count': df_20.nunique()
})
print(df_info)

           dtype  null_count  unique_count
country   object           0            22
1960     float64           0            22
1961     float64           0            20
1962     float64           0            21
1963     float64           0            21
...          ...         ...           ...
2016     float64           0            20
2017     float64           0            22
2018     float64           0            21
2019     float64           0            22
2020     float64           0            22

[62 rows x 3 columns]


In [1707]:
df_20.sample(10)

,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
23,United States,5.3,5.6,5.9,6.3,6.9,7.7,8.4,9.0,9.7,...,40.7,40.7,40.6,40.2,40.3,39.8,39.8,39.6,40.0,40.5
11,Latvia,11.9,12.3,12.4,12.3,12.8,13.3,12.6,12.3,12.1,...,44.6,45.0,44.6,44.0,41.5,40.9,40.4,39.5,38.4,39.5
17,Slovak Republic,4.7,4.4,4.6,4.7,5.0,5.3,5.3,5.7,5.9,...,34.0,35.4,37.0,38.9,39.2,40.2,40.1,40.0,40.1,40.7
2,Czech Republic,4.9,4.6,4.5,4.7,4.8,5.0,5.3,5.3,5.4,...,41.8,43.4,45.0,46.7,47.8,48.6,49.0,48.5,48.2,48.5
12,Luxembourg,3.2,3.4,3.1,3.1,3.2,3.7,3.2,3.5,3.2,...,34.1,37.1,37.8,39.1,38.8,40.7,40.8,39.5,40.4,41.6
20,Sweden,11.3,11.7,12.4,12.6,13.1,13.8,14.5,15.1,16.0,...,54.3,54.5,54.4,54.6,54.7,54.9,54.5,54.5,54.5,55.2
10,Italy,2.4,2.4,2.2,2.2,2.0,2.0,2.0,2.0,2.0,...,23.7,25.0,26.9,28.8,30.0,31.5,32.8,34.0,35.4,33.8
21,Switzerland,3.8,4.0,4.2,4.1,4.2,3.9,3.8,3.9,3.8,...,19.3,20.2,21.1,21.7,22.9,24.2,25.2,25.7,26.5,27.7
6,Greece,1.2,1.2,1.2,1.2,1.1,1.1,1.0,1.0,1.1,...,7.4,7.6,7.0,8.2,8.8,9.4,10.3,11.1,12.4,13.8
7,Hungary,5.5,5.5,5.4,5.3,5.2,5.2,5.1,5.0,5.0,...,42.3,44.5,45.6,47.3,47.9,46.7,44.7,43.9,38.7,30.4


In [1708]:
#df_20.to_csv('../data/Cleaned/cleaned_share_of_births_outside_of_marriage_oecd.csv', index=False)

In [1709]:
#df_20.to_sql('share_of_births_outside_of_marriage_oecd',engine, if_exists='replace', index=False)

In [1710]:
df_21_1= pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rate_mean_age_first_marriage_S1.csv')
#mean_age_first_marriage
df_21_1

,Country,Gender,1990,1991,1992,1993,1994,1995,1996,1997,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Australia,Male,"26,5","26,7","26,9","27,0","27,2","27,3","27,6","27,8",...,"29,7","29,8","29,9","30,0","30,1","30,3","30,4","30,7","30,7","30,6"
1,Australia,Female,"24,3","24,5","24,7","24,8","25,1","25,3","25,7","25,9",...,"28,0","28,1","28,3","28,4","28,5","28,7","28,8","29,2","29,3","29,2"
2,Czechia,Male,"24,3","24,3","24,5","24,7","25,1","25,5","25,9","26,5",...,"31,0","31,2","31,3","31,4","31,6","31,7","31,8","31,9","32,0","32,4"
3,Czechia,Female,"21,6","21,6","21,9","22,1","22,4","22,8","23,1","23,6",...,"28,1","28,3","28,5","28,7","28,8","29,0","29,1","29,2","29,4","29,7"
4,Denmark,Male,"30,5","30,6","31,0","31,4","31,8","31,9","32,5","32,2",...,"33,8","34,3","34,4","34,4","34,3","34,7","34,8","34,9","35,1","35,3"
5,Denmark,Female,"27,8","28,0","28,3","28,8","29,2","29,2","29,9","30,1",...,"31,4","31,8","31,9","31,9","31,9","32,2","32,4","32,5","32,8","33,0"
6,Greece,Male,"29,0","29,3","29,6","29,7","29,9","30,1","30,2","30,6",...,"32,7","32,8","32,9","33,0","33,2","33,2","33,3","33,4","33,7","33,8"
7,Greece,Female,"24,9","25,2","25,5","25,5","25,8","26,0","26,3","26,6",...,"29,4","29,5","29,7","29,9","30,1","30,1","30,3","30,3","30,7","30,7"
8,Japan,Male,"28,4","28,4","28,4","28,4","28,5","28,5","28,5","28,5",...,"30,7","30,8","30,9","31,1","31,1","31,1","31,1","31,1","31,2","31,0"
9,Japan,Female,"25,9","25,9","26,0","26,1","26,2","26,3","26,4","26,6",...,"29,0","29,2","29,3","29,4","29,4","29,4","29,4","29,4","29,6","29,4"


In [1711]:
df_info = pd.DataFrame({
    'datatypes': df_21_1.dtypes,
    'null_count': df_21_1.isnull().sum(),
    'unique_count': df_21_1.nunique()
})
print(df_info)

        datatypes  null_count  unique_count
Country    object           0            10
Gender     object           0             2
1990       object           0            17
1991       object           0            18
1992       object           0            18
1993       object           0            19
1994       object           0            16
1995       object           0            18
1996       object           0            19
1997       object           0            17
1998       object           0            14
1999       object           0            19
2000       object           0            18
2001       object           0            18
2002       object           0            19
2003       object           0            19
2004       object           0            16
2005       object           0            18
2006       object           0            18
2007       object           0            19
2008       object           0            18
2009       object           0   

In [1712]:
df_21_1.columns = df_21_1.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1713]:
# --- Ensure "country" and "gender" are strings
df_21_1["country"] = df_21_1["country"].astype(str).str.strip().str.title()
df_21_1["gender"] = df_21_1["gender"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_21_1.columns if c not in ["country", "gender"]]
# --- Robust cleaning -> convert to float ---
df_21_1[num_cols] = (
    df_21_1[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_21_1[num_cols] = df_21_1[num_cols].round(2)

In [1714]:
df_21_1.drop_duplicates(inplace=True)
df_21_1.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_21_1.dtypes,
    'null_count': df_21_1.isnull().sum(),
    'unique_count': df_21_1.nunique()
})
print(df_info)

        datatypes  null_count  unique_count
country    object           0             9
gender     object           0             2
1990      float64           0            15
1991      float64           0            16
1992      float64           0            16
1993      float64           0            17
1994      float64           0            15
1995      float64           0            16
1996      float64           0            17
1997      float64           0            15
1998      float64           0            13
1999      float64           0            17
2000      float64           0            16
2001      float64           0            16
2002      float64           0            17
2003      float64           0            17
2004      float64           0            15
2005      float64           0            17
2006      float64           0            17
2007      float64           0            17
2008      float64           0            16
2009      float64           0   

In [1715]:
#df_21_1.to_csv('../data/Cleaned/cleaned_mean_age_first_marriage_oecd.csv',index=False)

In [1716]:
#df_21_1.to_sql('mean_age_first_marriage_oecd', engine, if_exists='replace', index= False)

In [1717]:
df_21_2 = pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rates_S2.csv')
#divorce_rates_per_1000_oecd
df_21_2

,Country,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Austria,"1,4","1,3","1,3","1,3","1,4","1,4","1,5","1,5","1,6",...,"1,9","1,9","1,9","1,8","1,8","1,8","1,8","1,7","1,6","1,5"
1,Belgium,"0,7","0,7","0,8","0,9","1,0","1,1","1,3","1,3","1,4",...,"2,2","2,2","2,2","2,1","2,0","2,0","2,0","1,8","1,9","1,7"
2,Czechia,"2,2","2,4","2,3","2,5","2,5","2,6","2,5","2,5","2,6",...,"2,7","2,5","2,5","2,4","2,4","2,3","2,3","2,0","2,0","1,9"
3,Denmark,"1,9","2,7","2,6","2,5","2,6","2,6","2,6","2,6","2,6",...,"3,4","3,4","2,9","3,0","2,6","2,6","1,8","2,7","2,2","2,1"
4,Estonia,"3,2","3,2","3,3","3,2","3,3","3,4","3,6","3,9","3,8",...,"2,5","2,4","2,6","2,5","2,5","2,4","2,1","1,9",NaN,"1,9"
5,Finland,"1,3","1,6","1,8","1,9","2,1","2,0","2,1","2,1","2,2",...,"2,5","2,5","2,5","2,5","2,4","2,4","2,4","2,4","2,2","2,0"
6,Germany,"1,3","1,4","1,5","1,6","1,8","1,9","2,0","1,5","1,0",...,"2,1","2,1","2,0","2,0","1,9","1,8","1,8","1,7","1,7","1,6"
7,Greece,"0,4","0,4","0,4","0,5","0,4","0,4","0,4","0,5","0,5",...,"1,5","1,3","1,4","1,0","1,8",NaN,NaN,NaN,NaN,NaN
8,Hungary,"2,2","2,3","2,3","2,4","2,3","2,5","2,6","2,6","2,7",...,"2,0","2,0","2,1","2,0","1,9","1,7","1,8","1,5","1,9","1,8"
9,Italy,..,"0,3","0,6","0,3","0,3","0,2","0,2","0,2","0,2",...,"0,9","0,9","1,4","1,6","1,5","1,5","1,4","1,1","1,4","1,4"


In [1718]:
df_info = pd.DataFrame({
    'datatypes': df_21_2.dtypes,
    'null_count': df_21_2.isnull().sum(),
    'unique_count': df_21_2.nunique()
})
print(df_info)

        datatypes  null_count  unique_count
Country    object           0            28
1970       object           0            18
1971       object           0            19
1972       object           0            19
1973       object           0            18
1974       object           0            18
1975       object           0            19
1976       object           0            18
1977       object           0            18
1978       object           0            18
1979       object           0            15
1980       object           0            18
1981       object           0            20
1982       object           0            22
1983       object           0            24
1984       object           0            20
1985       object           0            19
1986       object           0            20
1987       object           0            20
1988       object           0            20
1989       object           0            19
1990       object           0   

In [1719]:
df_21_2.columns = df_21_2.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1720]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_21_2.columns if c != "country"]

df_21_2[num_cols] = (
    df_21_2[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [1721]:
df_21_2.drop_duplicates(inplace=True)
df_21_2.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_21_2.dtypes,
    'null_count': df_21_2.isnull().sum(),
    'unique_count': df_21_2.nunique()
})
print(df_info)

        datatypes  null_count  unique_count
country    object           0            23
1970      float64           0            15
1971      float64           0            17
1972      float64           0            15
1973      float64           0            14
1974      float64           0            15
1975      float64           0            16
1976      float64           0            14
1977      float64           0            13
1978      float64           0            15
1979      float64           0            12
1980      float64           0            14
1981      float64           0            17
1982      float64           0            17
1983      float64           0            19
1984      float64           0            16
1985      float64           0            15
1986      float64           0            16
1987      float64           0            16
1988      float64           0            15
1989      float64           0            15
1990      float64           0   

In [1722]:
df_21_2.head(8)

,country,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Austria,1.4,1.3,1.3,1.3,1.4,1.4,1.5,1.5,1.6,...,1.9,1.9,1.9,1.8,1.8,1.8,1.8,1.7,1.60,1.50
1,Belgium,0.7,0.7,0.8,0.9,1.0,1.1,1.3,1.3,1.4,...,2.2,2.2,2.2,2.1,2.0,2.0,2.0,1.8,1.90,1.70
2,Czechia,2.2,2.4,2.3,2.5,2.5,2.6,2.5,2.5,2.6,...,2.7,2.5,2.5,2.4,2.4,2.3,2.3,2.0,2.00,1.90
3,Denmark,1.9,2.7,2.6,2.5,2.6,2.6,2.6,2.6,2.6,...,3.4,3.4,2.9,3.0,2.6,2.6,1.8,2.7,2.20,2.10
5,Finland,1.3,1.6,1.8,1.9,2.1,2.0,2.1,2.1,2.2,...,2.5,2.5,2.5,2.5,2.4,2.4,2.4,2.4,2.20,2.00
6,Germany,1.3,1.4,1.5,1.6,1.8,1.9,2.0,1.5,1.0,...,2.1,2.1,2.0,2.0,1.9,1.8,1.8,1.7,1.70,1.60
8,Hungary,2.2,2.3,2.3,2.4,2.3,2.5,2.6,2.6,2.7,...,2.0,2.0,2.1,2.0,1.9,1.7,1.8,1.5,1.90,1.80
10,Japan,0.9,1.0,1.0,1.0,1.0,1.1,1.1,1.1,1.2,...,1.8,1.8,1.8,1.7,1.7,1.7,1.7,1.6,1.47,1.52


In [1723]:
#df_21_2.to_csv('../data/Cleaned/cleaned_divorce_rates_per_1000_oecd.csv', index=False)

In [1724]:
#df_21_2.to_sql('divorce_rates_per_1000_oecd',engine, if_exists= 'replace' , index=False)

In [1725]:
df_21_3= pd.read_csv('../data/Raw/OECD/SF_3_1_Marriage_divorce_rates_prev_marital_status_S3.csv')
#share_of_previous_marital_status
df_21_3

,Country,Previous marital status,2000,2001,2002,2003,2004,2005,2006,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Australia,Single never married,"75,9","76,1","75,5","75,6","76,2","76,9","77,3","78,2",...,"79,6","79,7","80,0","80,5","80,5","80,1","80,3","80,1","80,3","80,7"
1,Australia,Divorced,"22,0","21,8","22,4","22,3","21,8","21,3","20,9","20,2",...,"19,0","18,8","18,6","18,2","18,1","18,5","18,3","18,5","18,3","18,0"
2,Australia,Widowed,"2,1","2,1","2,1","2,1","1,9","1,8","1,8","1,7",...,"1,5","1,5","1,4","1,3","1,4","1,4","1,4","1,4","1,3","1,3"
3,Austria,Single never married,"76,6","74,7","74,1","73,7","72,9","73,1","73,9","74,8",...,"75,5","75,7","76,7","77,1","77,5","77,7","78,1","78,1","78,2","78,0"
4,Austria,Divorced,"22,2","24,2","24,7","25,2","25,9","25,7","24,9","24,2",...,"23,5","23,4","22,3","22,0","21,5","21,5","20,9","21,0","21,0","21,6"
5,Austria,Widowed,"1,2","1,1","1,2","1,1","1,2","1,2","1,1","1,0",...,"1,0","0,9","1,0","0,9","1,0","0,8","0,9","0,9","0,8","0,4"
6,Czechia,Single never married,"74,9","74,5","74,3","74,0","73,9","74,2","74,5","72,6",...,"74,0","74,0","75,2","75,6","76,6","76,7","76,4","76,4","76,1","75,9"
7,Czechia,Divorced,"23,7","24,2","24,4","24,7","24,7","24,5","24,4","26,1",...,"24,9","24,9","23,8","23,4","22,4","22,3","22,6","22,6","22,9","23,0"
8,Czechia,Widowed,"1,4","1,3","1,3","1,3","1,4","1,2","1,1","1,3",...,"1,2","1,1","1,0","1,0","1,0","1,0","1,0","1,0","1,0","1,1"
9,Denmark,Single never married,"75,9","76,0","76,2","76,4","76,0","75,6","75,6","76,3",...,"77,2","76,0","75,0","76,2","76,1","76,9","76,4","77,1","77,6","78,3"


In [1726]:
df_info = pd.DataFrame({
    'datatypes': df_21_3.dtypes,
    'null_count': df_21_3.isnull().sum(),
    'unique_count': df_21_3.nunique()
})
print(df_info)

                        datatypes  null_count  unique_count
Country                    object           0            20
Previous marital status    object           0             3
2000                       object           0            47
2001                       object           0            51
2002                       object           0            56
2003                       object           0            50
2004                       object           0            50
2005                       object           0            52
2006                       object           0            49
2008                       object           0            47
2009                       object           0            50
2010                       object           0            49
2011                       object           0            49
2012                       object           0            53
2013                       object           0            49
2014                       object       

In [1727]:
df_21_3.columns = df_21_3.columns.str.lower() \
                .str.replace(' ', '_') \
                .str.replace('(', '') \
                .str.replace(')', '') \
                .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_21_3.head()

,country,previous_marital_status,2000,2001,2002,2003,2004,2005,2006,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Australia,Single never married,"75,9","76,1","75,5","75,6","76,2","76,9","77,3","78,2",...,"79,6","79,7","80,0","80,5","80,5","80,1","80,3","80,1","80,3","80,7"
1,Australia,Divorced,"22,0","21,8","22,4","22,3","21,8","21,3","20,9","20,2",...,"19,0","18,8","18,6","18,2","18,1","18,5","18,3","18,5","18,3","18,0"
2,Australia,Widowed,"2,1","2,1","2,1","2,1","1,9","1,8","1,8","1,7",...,"1,5","1,5","1,4","1,3","1,4","1,4","1,4","1,4","1,3","1,3"
3,Austria,Single never married,"76,6","74,7","74,1","73,7","72,9","73,1","73,9","74,8",...,"75,5","75,7","76,7","77,1","77,5","77,7","78,1","78,1","78,2","78,0"
4,Austria,Divorced,"22,2","24,2","24,7","25,2","25,9","25,7","24,9","24,2",...,"23,5","23,4","22,3","22,0","21,5","21,5","20,9","21,0","21,0","21,6"


In [1728]:
# --- Ensure "country" and "previous_marital_status" are strings
df_21_3["country"] = df_21_3["country"].astype(str).str.strip().str.title()
df_21_3["previous_marital_status"] = df_21_3["previous_marital_status"].astype(str).str.strip()

# --- Identify numeric columns: everything except country and birth order ---
num_cols = [c for c in df_21_3.columns if c not in ["country", "previous_marital_status"]]
# --- Robust cleaning -> convert to float ---
df_21_3[num_cols] = (
    df_21_3[num_cols].astype(str)
    .replace({"\xa0": "", "\u202f": "", ",": "."}, regex=True)
    .replace(r"[^\d\.\-]", "", regex=True)
    .replace(r"\.\.+", ".", regex=True)
    .replace(r"^\.$|^\s*$", np.nan, regex=True)
    .apply(pd.to_numeric, errors="coerce")
)
df_21_3[num_cols] = df_21_3[num_cols].round(2)

In [1729]:
df_21_3.drop_duplicates(inplace=True)
df_21_3.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_21_3.dtypes,
    'null_count': df_21_3.isnull().sum(),
    'unique_count': df_21_3.nunique()
})
print(df_info)

                        datatypes  null_count  unique_count
country                    object           0            20
previous_marital_status    object           0             3
2000                      float64           0            47
2001                      float64           0            51
2002                      float64           0            56
2003                      float64           0            50
2004                      float64           0            50
2005                      float64           0            52
2006                      float64           0            49
2008                      float64           0            47
2009                      float64           0            50
2010                      float64           0            49
2011                      float64           0            49
2012                      float64           0            53
2013                      float64           0            49
2014                      float64       

In [1730]:
#df_21_3.to_csv('../data/Cleaned/cleaned_share_of_previous_marital_status_oecd.csv', index=False)

In [1731]:
#df_21_3.to_sql('share_of_previous_marital_status_oecd', engine, if_exists= 'replace', index =  False)

In [1732]:
df_22_1 = pd.read_csv('../data/Raw/OECD/SF3_3_A_in_private_households_by_partnership_status_S1.csv')
#hauseholds_by_partnership_status_oecd
df_22_1

,Country,20+_All_Total_Living_with_a_partner(%),20+_All_Married or in a civil or registered partnership_living_with_a_partner(%),20+_All_Cohabiting_living_with_a_partner(%),20+_All_Not living with a partner(%),20/34_Total_living_with_a_partner(%),20/34_Married or in a civil or registered partnership_living_with_a_partner(%),20/34_Cohabiting_living_with_a_partner(%),Not living with a partner_Total(%),Living with at least one parent(%)
0,Australia (c),"63,79","53,59","10,20","36,21","47,06","29,41","17,65","52,94",..
1,Austria,"58,80","49,10","9,70","41,20","39,11","22,15","16,97","60,89","33,82"
2,Belgium,"62,15","53,51","8,64","37,85","45,28","29,33","15,94","54,72","31,34"
3,Canada (d),"66,89","54,46","12,43","33,11","55,34","33,55","21,79","44,66",..
4,Czech Republic,"51,17","45,39","5,79","48,83","30,78","21,32","9,46","69,22","36,20"
5,Denmark,"64,15","50,02","14,12","35,85","50,54","21,86","28,68","49,46","10,67"
6,Estonia,"53,93","37,30","16,64","46,07","45,31","17,81","27,50","54,69","26,46"
7,France,"64,14","49,41","14,72","35,86","50,42","21,89","28,53","49,58","22,08"
8,Germany,"62,61","53,91","8,69","37,39","39,53","22,15","17,39","59,74","27,54"
9,Greece,"60,23","58,52","1,71","39,77","33,13","29,24","3,90","66,87","45,43"


In [1733]:
df_info = pd.DataFrame({
    'datatypes': df_22_1.dtypes,
    'null_count': df_22_1.isnull().sum(),
    'unique_count': df_22_1.nunique()
})
print(df_info)

                                                   datatypes  null_count  \
Country                                               object           0   
20+_All_Total_Living_with_a_partner(%)                object           0   
20+_All_Married or in a civil or registered par...    object           0   
20+_All_Cohabiting_living_with_a_partner(%)           object           0   
20+_All_Not living with a partner(%)                  object           0   
20/34_Total_living_with_a_partner(%)                  object           0   
20/34_Married or in a civil or registered partn...    object           0   
20/34_Cohabiting_living_with_a_partner(%)             object           0   
Not living with a partner_Total(%)                    object           0   
Living with at least one parent(%)                    object           0   

                                                    unique_count  
Country                                                       37  
20+_All_Total_Living_with_a_p

In [1734]:
df_22_1.columns = df_22_1.columns.str.lower() \
                .str.replace(' ', '_') \


df_22_1.head()

,country,20+_all_total_living_with_a_partner(%),20+_all_married_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),20+_all_cohabiting_living_with_a_partner(%),20+_all_not_living_with_a_partner(%),20/34_total_living_with_a_partner(%),20/34_married_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),20/34_cohabiting_living_with_a_partner(%),not_living_with_a_partner_total(%),living_with_at_least_one_parent(%)
0,Australia (c),"63,79","53,59","10,20","36,21","47,06","29,41","17,65","52,94",..
1,Austria,"58,80","49,10","9,70","41,20","39,11","22,15","16,97","60,89","33,82"
2,Belgium,"62,15","53,51","8,64","37,85","45,28","29,33","15,94","54,72","31,34"
3,Canada (d),"66,89","54,46","12,43","33,11","55,34","33,55","21,79","44,66",..
4,Czech Republic,"51,17","45,39","5,79","48,83","30,78","21,32","9,46","69,22","36,20"


In [1735]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_22_1.columns if c != "country"]

df_22_1[num_cols] = (
    df_22_1[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [1736]:
df_22_1["country"] = df_22_1["country"].str.replace(r"\s*\(.*?\)", "", regex=True)
print(df_22_1["country"].unique())

['Australia' 'Austria' 'Belgium' 'Canada' 'Czech Republic' 'Denmark'
 'Estonia' 'France' 'Germany' 'Greece' 'Hungary' 'Iceland' 'Ireland'
 'Italy' 'Latvia' 'Luxembourg' 'Netherlands' 'New Zealand' 'Norway'
 'Poland' 'Portugal' 'Slovak Republic' 'Slovenia' 'Spain' 'Sweden'
 'Switzerland' 'United Kingdom' 'United States' 'OECD-28 average'
 'Bulgaria' 'Croatia' 'Cyprus' 'Lithuania' 'Malta' 'Romania' 'EU average'
 'Eurozone average']


In [1737]:
df_22_1.drop_duplicates(inplace = True)
df_22_1.dropna(inplace=True)

df_info = pd.DataFrame({
    'datatypes': df_22_1.dtypes,
    'null_count': df_22_1.isnull().sum(),
    'unique_count': df_22_1.nunique()
})
print(df_info)

                                                   datatypes  null_count  \
country                                               object           0   
20+_all_total_living_with_a_partner(%)               float64           0   
20+_all_married_or_in_a_civil_or_registered_par...   float64           0   
20+_all_cohabiting_living_with_a_partner(%)          float64           0   
20+_all_not_living_with_a_partner(%)                 float64           0   
20/34_total_living_with_a_partner(%)                 float64           0   
20/34_married_or_in_a_civil_or_registered_partn...   float64           0   
20/34_cohabiting_living_with_a_partner(%)            float64           0   
not_living_with_a_partner_total(%)                   float64           0   
living_with_at_least_one_parent(%)                   float64           0   

                                                    unique_count  
country                                                       34  
20+_all_total_living_with_a_p

In [1738]:
df_22_1.sample(8)

,country,20+_all_total_living_with_a_partner(%),20+_all_married_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),20+_all_cohabiting_living_with_a_partner(%),20+_all_not_living_with_a_partner(%),20/34_total_living_with_a_partner(%),20/34_married_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),20/34_cohabiting_living_with_a_partner(%),not_living_with_a_partner_total(%),living_with_at_least_one_parent(%)
18,Norway,60.96,46.07,14.89,39.04,41.99,18.94,23.05,58.01,25.54
19,Poland,57.72,55.60,2.12,42.28,37.60,34.20,3.40,62.40,45.65
14,Latvia,47.87,38.55,9.32,52.13,33.13,20.71,12.41,66.87,41.78
8,Germany,62.61,53.91,8.69,37.39,39.53,22.15,17.39,59.74,27.54
25,Switzerland,64.15,53.46,10.69,35.85,42.43,25.40,17.03,55.00,31.44
16,Netherlands,66.82,53.10,13.72,33.18,46.76,21.40,25.35,53.23,23.70
12,Ireland,58.99,50.23,8.75,41.01,37.80,21.40,16.40,57.25,28.18
23,Spain,61.43,52.55,8.88,38.57,38.43,22.51,15.92,61.57,45.02


In [1739]:
#df_22_1.to_csv('../data/Cleaned/cleaned_hauseholds_by_partnership_status_oecd.csv', index=False)

In [1740]:
#df_22_1.to_sql('hauseholds_by_partnership_status_oecd', engine, if_exists='replace', index= False)

In [1741]:
df_22_2 = pd.read_csv('../data/Raw/OECD/SF3_3_B_ by level of educational attainment_S2.csv')
#level_of_educational_attainment
df_22_2

,Country,Low_Education_Total_living_with_a_partner(%),Low_educationMarried or in a civil or registered partnership_living_with_a_partner(%),Low_education_Cohabiting_living_with_a_partner(%),Not living with a partner_Low_education(%),Medium education_Total_Living with a partner(%),Medium education_Married or in a civil or registered partnership_Living with a partner(%),Medium education_Cohabiting_Living with a partner(%),Not living with a partner_Medium education(%),High education_Total_Living with a partner(%),High education_Married or in a civil or registered partnership_Living with a partner(%),High education_Cohabiting_Living with a partner(%),Not living with a partner_High education(%)
0,Austria,"56,81","50,49","6,32","43,19","59,27","48,73","10,54",NaN,"60,03","48,38","11,65","39,97"
1,Belgium,"62,28","56,11","6,17","37,72","60,79","49,80","10,99",NaN,"67,09","56,58","10,51","32,91"
2,Czech Republic,"40,81","36,55","4,26","59,19","53,99","47,87","6,12","46,01","57,29","50,26","7,03","42,71"
3,Estonia,"42,17","26,39","15,78","57,83","54,41","36,61","17,79","45,59","60,14","44,45","15,69","39,86"
4,France,"61,12","51,93","9,18","38,88","65,68","49,17","16,51","34,32","65,58","46,60","18,98","34,42"
5,Germany,"54,46","48,79","5,67","45,54","62,38","53,13","9,25","37,62","68,89","59,16","9,74","31,11"
6,Greece,"63,81","62,88","0,93","36,19","57,00","54,88","2,12","43,00","58,33","55,70","2,63","41,67"
7,Hungary,"50,33","40,38","9,95","49,67","57,94","46,78","11,15","42,06","59,56","51,02","8,55","40,44"
8,Iceland,"51,86","41,02","10,84","48,14","58,31","46,57","11,74","41,69","69,72","54,53","15,19","30,28"
9,Latvia,"36,27","25,92","10,35","63,73","49,32","39,54","9,78","50,68","52,91","45,39","7,52","47,09"


In [1742]:
df_info = pd.DataFrame({
    'datatypes': df_22_2.dtypes,
    'null_count': df_22_2.isnull().sum(),
    'unique_count': df_22_2.nunique()
})
print(df_info)

                                                   datatypes  null_count  \
Country                                               object           0   
Low_Education_Total_living_with_a_partner(%)          object           0   
Low_educationMarried or in a civil or registere...    object           0   
Low_education_Cohabiting_living_with_a_partner(%)     object           0   
Not living with a partner_Low_education(%)            object           0   
Medium education_Total_Living with a partner(%)       object           0   
Medium education_Married or in a civil or regis...    object           0   
Medium education_Cohabiting_Living with a partn...    object           0   
Not living with a partner_Medium education(%)         object           2   
High education_Total_Living with a partner(%)         object           0   
High education_Married or in a civil or registe...    object           0   
High education_Cohabiting_Living with a partner(%)    object           0   
Not living w

In [1743]:
df_22_2.columns = df_22_2.columns.str.lower() \
                .str.replace(' ', '_') \


df_22_2.head()

,country,low_education_total_living_with_a_partner(%),low_educationmarried_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),low_education_cohabiting_living_with_a_partner(%),not_living_with_a_partner_low_education(%),medium_education_total_living_with_a_partner(%),medium_education_married_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),medium_education_cohabiting_living_with_a_partner(%),not_living_with_a_partner_medium_education(%),high_education_total_living_with_a_partner(%),high_education_married_or_in_a_civil_or_registered_partnership_living_with_a_partner(%),high_education_cohabiting_living_with_a_partner(%),not_living_with_a_partner_high_education(%)
0,Austria,"56,81","50,49","6,32","43,19","59,27","48,73","10,54",NaN,"60,03","48,38","11,65","39,97"
1,Belgium,"62,28","56,11","6,17","37,72","60,79","49,80","10,99",NaN,"67,09","56,58","10,51","32,91"
2,Czech Republic,"40,81","36,55","4,26","59,19","53,99","47,87","6,12","46,01","57,29","50,26","7,03","42,71"
3,Estonia,"42,17","26,39","15,78","57,83","54,41","36,61","17,79","45,59","60,14","44,45","15,69","39,86"
4,France,"61,12","51,93","9,18","38,88","65,68","49,17","16,51","34,32","65,58","46,60","18,98","34,42"


In [1744]:
# Convert all but 'country' to float (robust + compact)
num_cols = [c for c in df_22_2.columns if c != "country"]

df_22_2[num_cols] = (
    df_22_2[num_cols].astype(str)
    .replace({'\xa0': '', '\u202f': '', ',': '.'}, regex=True)   # spaces & decimal comma
    .replace(r'[^\d\.\-]', '', regex=True)                       # keep digits/dot/minus
    .replace(r'\.\.+', '.', regex=True)                          # collapse multi-dots
    .replace(r'^\.$|^\s*$', np.nan, regex=True)                  # lone dot/empty -> NaN
    .apply(pd.to_numeric, errors="coerce")
)

In [1745]:
df_22_2["country"] = df_22_2["country"].str.replace(r"\s*\(.*?\)", "", regex=True)
print(df_22_2["country"].unique())

['Austria' 'Belgium' 'Czech Republic' 'Estonia' 'France' 'Germany'
 'Greece' 'Hungary' 'Iceland' 'Latvia' 'Luxembourg' 'Norway' 'Poland'
 'Portugal' 'Slovenia' 'Spain' 'Sweden' 'United Kingdom' 'OECD-19 average'
 'Bulgaria' 'Croatia' 'Cyprus' 'Lithuania' 'Malta' 'Romania' 'EU average'
 'Eurozone average']


In [1746]:
df_22_2.drop_duplicates(inplace=True)
df_22_2.dropna(inplace=True)


df_info = pd.DataFrame({
    'datatypes': df_22_2.dtypes,
    'null_count': df_22_2.isnull().sum(),
    'unique_count': df_22_2.nunique()
})
print(df_info)

                                                   datatypes  null_count  \
country                                               object           0   
low_education_total_living_with_a_partner(%)         float64           0   
low_educationmarried_or_in_a_civil_or_registere...   float64           0   
low_education_cohabiting_living_with_a_partner(%)    float64           0   
not_living_with_a_partner_low_education(%)           float64           0   
medium_education_total_living_with_a_partner(%)      float64           0   
medium_education_married_or_in_a_civil_or_regis...   float64           0   
medium_education_cohabiting_living_with_a_partn...   float64           0   
not_living_with_a_partner_medium_education(%)        float64           0   
high_education_total_living_with_a_partner(%)        float64           0   
high_education_married_or_in_a_civil_or_registe...   float64           0   
high_education_cohabiting_living_with_a_partner(%)   float64           0   
not_living_w

In [1747]:
#df_22_2.to_csv('../data/Cleaned/cleaned_level_of_educational_attainment_oecd.csv', index=False)

In [1748]:
#df_22_2.to_sql('level_of_educational_attainment_oecd',engine, if_exists='replace', index= False)

In [1749]:
df_6666 = pd.read_csv('../data/Raw/OECD/OECD_df_famliy_selected.csv')
df_6666

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,COU,Country,SEX,Sex,IND,Indicator,...,OBS_VALUE,Observation Value,OBS_STATUS,Observation Status,UNIT_MEASURE,Unit of Measures,UNIT_MULT,Multiplier,BASE_PER,Base reference period
0,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,LVA,Latvia,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,39.5,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
1,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,GRC,Greece,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,11.1,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
2,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,CHL,Chile,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,74.8,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
3,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,NLD,Netherlands,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,51.9,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
4,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,LTU,Lithuania,TOTAL,Total,FAM3,Share of births outside of marriage (% of all ...,...,26.4,NaN,A,NaN,PC,Percentage,0,Units,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,COL,Colombia,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.4,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
501,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.5,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
502,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.6,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
503,DATAFLOW,OECD:DF_FAMILY(1.0),Family Database,I,OAVG,OECD - Average,TOTAL,Total,FAM2,Mean age of women at childbirth,...,30.7,NaN,A,NaN,YR,Years,0,Units,NaN,NaN
